In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget https://www.cis.uni-muenchen.de/~poerner/blobs/e-bert/wikipedia2vec-base-cased.bert-base-cased.linear.npy

--2021-01-28 17:20:50--  https://www.cis.uni-muenchen.de/~poerner/blobs/e-bert/wikipedia2vec-base-cased.bert-base-cased.linear.npy
Resolving www.cis.uni-muenchen.de (www.cis.uni-muenchen.de)... 129.187.148.72, 2001:4ca0:4f01::5
Connecting to www.cis.uni-muenchen.de (www.cis.uni-muenchen.de)|129.187.148.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2359424 (2.2M)
Saving to: ‘wikipedia2vec-base-cased.bert-base-cased.linear.npy’

wikipedia2vec-base- 100%[===================>]   2.25M  1.35MB/s    in 1.7s    

2021-01-28 17:20:53 (1.35 MB/s) - ‘wikipedia2vec-base-cased.bert-base-cased.linear.npy’ saved [2359424/2359424]



In [ ]:
%%writefile prep.sh

wget https://allennlp.s3-us-west-2.amazonaws.com/knowbert/wiki_entity_linking/aida_train.txt
wget https://allennlp.s3-us-west-2.amazonaws.com/knowbert/wiki_entity_linking/aida_test.txt
wget https://allennlp.s3-us-west-2.amazonaws.com/knowbert/wiki_entity_linking/aida_dev.txt
pip install transformers
pip install seqeval

pip install wikipedia2vec

git clone https://github.com/allenai/kb
cd kb
pip install -r requirements.txt
python -c "import nltk; nltk.download('wordnet')"
python -m spacy download en_core_web_sm
pip install --editable .
cd ../..

Writing prep.sh


In [ ]:
!sh prep.sh


--2021-01-28 18:01:43--  https://allennlp.s3-us-west-2.amazonaws.com/knowbert/wiki_entity_linking/aida_train.txt
Resolving allennlp.s3-us-west-2.amazonaws.com (allennlp.s3-us-west-2.amazonaws.com)... 52.218.154.97
Connecting to allennlp.s3-us-west-2.amazonaws.com (allennlp.s3-us-west-2.amazonaws.com)|52.218.154.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1837710 (1.8M) [text/plain]
Saving to: ‘aida_train.txt’

aida_train.txt      100%[===================>]   1.75M  2.21MB/s    in 0.8s    

2021-01-28 18:01:45 (2.21 MB/s) - ‘aida_train.txt’ saved [1837710/1837710]

--2021-01-28 18:01:45--  https://allennlp.s3-us-west-2.amazonaws.com/knowbert/wiki_entity_linking/aida_test.txt
Resolving allennlp.s3-us-west-2.amazonaws.com (allennlp.s3-us-west-2.amazonaws.com)... 52.218.154.97
Connecting to allennlp.s3-us-west-2.amazonaws.com (allennlp.s3-us-west-2.amazonaws.com)|52.218.154.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 431176 

In [ ]:
!git clone https://github.com/NPoe/ebert.git

Cloning into 'ebert'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 48 (delta 14), reused 24 (delta 0), pack-reused 0
Unpacking objects: 100% (48/48), done.


In [ ]:
import sys

sys.path += ['ebert/code']
sys.path += ['kb']
print(sys.path)


from tqdm import tqdm
import numpy as np
import random
from transformers import AutoTokenizer, BertModel, AdamW,PreTrainedTokenizerFast
import torch
from seqeval.metrics import f1_score, classification_report, accuracy_score
from collections import OrderedDict
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance
from embeddings import MappedEmbedding, load_embedding
from mappers import load_mapper



['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', 'entity-recognition-datasets/src', 'ebert/code', 'kb']


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", do_lower_case = False, use_fast=True)
assert isinstance(tokenizer, PreTrainedTokenizerFast)

left_pattern = ["[MASK]", "/"]
right_pattern = ["*"]
ent_prefix = "ENTITY/"
granularity = "document"
max_len = 512

In [ ]:
ent_prefix = "ENTITY/"
ebert_name = "drive/My Drive/wikipedia2vec-base-cased"
ebert_emb = load_embedding(ebert_name, prefix = ent_prefix)
ebert_emb = MappedEmbedding(ebert_emb, load_mapper('wikipedia2vec-base-cased.bert-base-cased.linear.npy'))

In [ ]:
# based of https://github.com/NPoe/ebert
def read_aida_file(f, ignore_gold = True):

  data, sentence, spans = {}, [], []
  flag = False
  doc_count = -1

  with open(f) as handle:
      for line in handle:
          line = line.strip()
          if len(line) == 0: continue

          if line.startswith("DOCSTART"):
              doc_count += 1
              in_doc_count = -1
          if (line == "*NL*" and granularity == "paragraph") or line == "DOCEND":
              if len(sentence):
                  in_doc_count += 1
                  if ignore_gold:
                      data[(doc_count, in_doc_count)] = [sentence, []]
                  else:
                      data[(doc_count, in_doc_count)] = [sentence, spans]
              sentence, spans = [], []
          elif line == "*NL*":
              pass
          elif line.startswith("DOCSTART"):
              doc_count += 1
          elif line.startswith("MMEND"):
              flag = False
          elif flag:
              spans[-1][-1] += 1
              sentence.append(line.split()[0])
          elif line.startswith("MMSTART"):
              assert not flag
              gold_entity = line.split()[-1]
              spans.append([gold_entity, len(sentence), len(sentence)-1])
              flag = True
          else: 
              sentence.append(line.split()[0])

  if 1:
      delete, checked = set(), set()

      while len(delete) + len(checked) != len(data):
          for data_idx, (sentence, spans) in data.items():
              if (not data_idx in delete) and (not data_idx in checked):
                  if len(tokenizer.tokenize(" ".join(sentence + left_pattern + right_pattern))) >= max_len-2:
                      midpoint = len(sentence) // 2
                      breaking = False
                      for i in range(0, midpoint - 5):
                          if breaking: break
                          for direction in (-1, 1):
                              point = midpoint + (direction * i)
                              if sentence[point] == "." and not any([x[1] <= point+1 and x[2] >= point+1 for x in spans]):
                                  midpoint = point+1
                                  breaking = True
                                  break

                      sentence_a, sentence_b = sentence[:midpoint], sentence[midpoint:]
              
                      spans_a = [x for x in spans if x[2] < midpoint]
                      spans_b = [(x[0], x[1]-midpoint, x[2]-midpoint) for x in spans if x[2] > midpoint]
                  
                      data[data_idx + (1,)] = [sentence_a, spans_a]
                      data[data_idx + (2,)] = [sentence_b, spans_b]
                      delete.add(data_idx)
                      break

                  else:
                      checked.add(data_idx)

      assert len(delete.intersection(checked)) == 0
      for data_idx in delete:
          del data[data_idx]

  return data

# # use this to prepare and load 
# def getNeedEntities(data,gold_candidate_set,candidate_set,candidate_generator):
#   for data_idx, (sentence, spans) in tqdm(data.items(), desc = "iterating over data"):
#     for entity, start, end in spans:
#       gold_candidate_set.add(entity)

#       # Getting all possible candidates for the right spans
#       if not candidate_generator is None:
#         for candidate_list in candidate_generator.get_mentions_raw_text(" ".join(sentence[start:end+1]))['candidate_entities']:
#           candidate_set.update(candidate_list)   

# use this to prepare and load 


def get_needed_entity_vectors(model,candidate_set):
  entity_vector_dict = dict()
  # ones we didnt find 
  errors = []
  for candidate in candidate_set:
    # print(candidate)
    try:
      candidate_p = " ".join(candidate.split("_"))
      # candidate_p = candidate_p.lower()
      entity_vector_dict[candidate] = model.mapper.apply(model.embedding.model.get_entity_vector(candidate_p))
    except KeyError:
      errors.append(candidate)

  return entity_vector_dict,errors



# for tokens and iob style labels, perform WordPiece tokenization and pad ##tokens labels with 'X' 
# TODO X or same label as head
# def extend_labels_WordPiece(sentence,labels,tokenizer):
#   tokens = [] 
#   wp_labels = []
#   for i, word in enumerate(sentence):
#     tokenized = tokenizer.tokenize(word)
#     tokens.extend(tokenized)
#     for m,token in enumerate(tokenized):

#       # head token with the original label
#       if(m == 0):
#         wp_labels.append(labels[i])

#       # tail tokens with ##
#       else:
#         wp_labels.append('X')
#   tokens = ["[CLS]"] + tokens + ["[SEP]"]
#   wp_labels = ["X"] + wp_labels + ["X"]
#   return tokens, wp_labels

# label_dict = {'O':0, 'B':1, 'I':2, 'X':3}

# # this also pads labels to max_len
# def labels_to_tensor(labels,max_len):
#   tensor_labels = []
#   for example_labels in labels:
#     transformed = [label_dict[label] for label in example_labels]

#     # add padding if required
#     if(len(transformed) < max_len):
#       transformed = transformed + [0] * (max_len-len(transformed))

#     tensor_labels.append(transformed)

#   return torch.LongTensor(tensor_labels)


class Sample:
  def __init__(self, sentence,iob_labels,entities,spans):
      # tokenized
      self.sentence = sentence

      self.iob_labels = iob_labels

      # golden entities for B tag in NER (iob) labels
      self.entities = entities

      # spans for the golden entities
      self.spans = spans

      self.ed_labels = None

  def add_ed_labels(self,ed_labels):
    self.ed_labels = ed_labels
      
def get_samples(data):
  samples = []
  for data_idx, (sentence, spans) in tqdm(data.items(), desc = "Converting data to samples"):
    iob_labels = list('O' * len(sentence))
    entities = []
    sample_entity_spans = []
    for entity, start, end in spans:

      #getting all possible candidates for the right spans
      # print(" ".join(sentence[start:end+1]))     


      # iob labeling
      for i in range(start, end+1):
        if(i == start):
          iob_labels[i] = 'B'
        else:
          iob_labels[i] = 'I'
      entities.append(entity)
      sample_entity_spans.append((start,end))
    # tokens, wp_labels = extend_labels_WordPiece(sentence,iob_labels,tokenizer)
    # assert len(sentence) == len(iob_labels)
    # assert len(tokens) == len(wp_labels)
    # data[data_idx].append(tokens)
    # data[data_idx].append(iob_labels,)
    samples.append(Sample(sentence,iob_labels,entities,sample_entity_spans))

  return samples

In [ ]:
train_data = read_aida_file('aida_train.txt', ignore_gold = False)
val_data = read_aida_file('aida_dev.txt', ignore_gold = False)
test_data = read_aida_file('aida_test.txt', ignore_gold = False)

train_samples = get_samples(train_data)
val_samples = get_samples(val_data)
test_samples = get_samples(test_data)

Token indices sequence length is longer than the specified maximum sequence length for this model (553 > 512). Running this sequence through the model will result in indexing errors
Converting data to samples: 100%|██████████| 265/265 [00:00<00:00, 46302.46it/s]


In [ ]:
max_size = 0
for key in train_data.keys():
  if len(train_data[key][0]) > max_size:
    max_size = len(train_data[key][0])
max_size

464

In [ ]:
" ".join(train_data[(321, 0, 1, 2, 2, 2)][0])

"Kenny Harrison ( U.S. ) 16.97 5. Gennadi Markov ( Russia ) 16.66 6. Francis Agyepong ( Britain ) 16.63 7. Rogel Nachum ( Israel ) 16.36 8. Sigurd Njerve ( Norway ) 16.35 Men 's 1,500 metres 1. Hicham El Guerrouj ( Morocco ) three minutes 29.05 seconds 2. Isaac Viciosa ( Spain ) 3:33.00 3. William Tanui ( Kenya ) 3:33.36 4. Elijah Maru ( Kenya ) 3:33.64 5. Marcus O'Sullivan ( Ireland ) 3:33.77 6. John Mayock ( Britain ) 3:33.94 7. Laban Rotich ( Kenya ) 3:34.12 8. Christophe Impens ( Belgium ) 3:34.13 Women 's high jump 1. Stefka Kostadinova ( Bulgaria ) 2.03 metres 2. Inga Babakova ( Ukraine ) 2.03 3. Alina Astafei ( Germany ) 1.97 4. Tatyana Motkova ( Russia ) 1.94 5. Svetlana Zalevskaya ( Kazakhstan ) 1.91 6. Yelena Gulyayeva ( Russia ) 1.88 7. Hanna Haugland ( Norway ) 1.88 8 equal ."

In [ ]:
from wikipedia2vec import Wikipedia2Vec

wiki2vec = Wikipedia2Vec.load('drive/My Drive/wikipedia2vec-base-cased')

In [ ]:
# Tokenize all texts and align the labels with them.
label_dict = {'O':0, 'B':1, 'I':2}
int_to_label_list = ['O', 'B', 'I']
def tokenize_and_align_labels(sentences,iob_labels,tokenizer,label_all_tokens = False):
    tokenized_inputs = tokenizer(
        sentences,
        padding='max_length',
        max_length = max_len,
        truncation=True,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
        return_tensors="pt"
    )
    labels = []
    for i, label in enumerate(iob_labels):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label_dict[label[word_idx]])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


def eval_preds(labels,preds):
  true_predictions = [
      [int_to_label_list[p] for (p, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(preds, labels)
  ]
  true_labels = [
      [int_to_label_list[l] for (p, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(preds, labels)
  ]
  print()
  print(f1_score(true_labels,true_predictions))
  print(classification_report(true_labels, true_predictions))
  print()

In [ ]:
def get_aligned_entity_labels(aligned_ner_labels,entities,entity_to_int_map):

  aligned_entity_labels = []
  for sample_ner_labels,sample_entities in zip(aligned_ner_labels,entities):
    sample_entity_labels = []

    entity_index = 0
    for label in sample_ner_labels:
    
      # the loss is only calculated for B tags (first head token in a mention)
      if (label == 1):
        
        sample_entity_labels.append(entity_to_int_map[sample_entities[entity_index]])
        entity_index += 1

      else:

        # this will be ignored for loss
        sample_entity_labels.append(-100)

    
    aligned_entity_labels.append(sample_entity_labels)

  return aligned_entity_labels


# todo remove map
def get_entity_vectors(ed_labels,entity_vector_dict,int_to_entity_map,entity_dim=768):
  entity_vectors = torch.zeros((ed_labels.shape[0],ed_labels.shape[1],entity_dim),dtype=torch.float32)
  for i,sample_labels in enumerate(ed_labels):
    for j,label in enumerate(sample_labels):
      if(label != -100):
        # print(label.cpu().numpy())
        entity_vectors[i][j] = torch.tensor(entity_vector_dict[int_to_entity_map[label.item()]])
  return entity_vectors

# TODO do this once before training
# def get_entity_locations(labels):
#   entity_locations = []
#   counter = 0
#   for i in range(len(labels)):
#     lst = []
#     for j in range(len(labels[i])):
#       if(labels[i][j] != -100):
#         lst.append(j)
#         counter+=1
#     entity_locations.append(lst)
#   return entity_locations,counter


#TODO do this once before training
def get_entity_locations(labels):
  entity_locations = []
  counter = 0
  for i in range(len(labels)):
    lst = []
    span_size = 0
    span_beginning = None
    for j in range(len(labels[i])):

      if (labels[i][j] == 1):
        counter+=1

        # empty the previous span 
        if (span_size > 0 ):
          lst.append((span_beginning,span_size))

        span_beginning = j
        span_size = 1

      elif (labels[i][j] == 2):
        span_size +=1


      elif (labels[i][j] == 0):
        if (span_size > 0):
          lst.append((span_beginning,span_size))
          span_size = 0
        
    if (span_size > 0 ):
      lst.append((span_beginning,span_size))
      span_size = 0

    ########### make sure we empty queue ############

    entity_locations.append(lst)
  return entity_locations,counter

  

def get_dataset_entity_candidates(samples,candidate_generator,model,MAX_CANDIDATES=20):
  dataset_candidates = []
  spans_to_eval =[]
  for sample in samples:
    sample_spans = [ " ".join(sample.sentence[start:end+1]) for start,end in sample.spans]
    spans_to_eval.extend(sample_spans)

  for span in spans_to_eval:
    generator_output = candidate_generator.get_mentions_raw_text(span)

    candidates = []
    for i in range(len(generator_output['candidate_spans'])):
      candidates.extend(zip(generator_output['candidate_entities'][i],generator_output['candidate_entity_priors'][i]))
    candidates = sorted(candidates, key=lambda x: x[1],reverse=True)

    # titles_only = [x[0] for x in candidates]
    filtered = filter_non_available_entities(candidates,model)
    span_candidates = list(OrderedDict.fromkeys(filtered))
    # print(titles_only)
    if(len(span_candidates) > MAX_CANDIDATES ):
      span_candidates = span_candidates[:MAX_CANDIDATES]
    if(len(span_candidates) == 0):
      print('No candidates for: ', span)
    dataset_candidates.append(span_candidates)
  return dataset_candidates


def get_candidates_vectors(entities):
  vectors = []
  for entity,prior in entities:
    entity_name = " ".join(entity.split("_"))
    # vector = wiki2vec.get_entity_vector(entity_name)
    vector = ebert_emb.mapper.apply(ebert_emb.embedding.model.get_entity_vector(entity_name))
    vectors.append(vector)
  return vectors

def filter_non_available_entities(entities,model):
  filtered_list = []
  for entity,prior in entities:
    entity_name = " ".join(entity.split("_"))
    try:
      model.mapper.apply(model.embedding.model.get_entity_vector(entity_name))
      filtered_list.append((entity,prior))
    except:
      # print("couldn't find ", entity)
      'Do nothing TODO'
  return filtered_list


def ED_avg_cos_dis(eval_entity_labels,eval_ed_outputs):
  # a list with predicted vectors
  predicted_vectors = []
  true_labels = []
  true_vectors = []
  #TODO all sets
  for i in range(len(eval_entity_labels)):
    for j in range(max_len):
      if(eval_entity_labels[i][j] != -100):
        true_labels.append(int_to_entity_map[eval_entity_labels[i][j]])
        true_vectors.append(entity_vector_dict[int_to_entity_map[eval_entity_labels[i][j]]])
        predicted_vectors.append(eval_ed_outputs[i][j].cpu().numpy())
  distances = -100* np.ones(len(true_labels))
  for i in range(len(true_labels)):
    distances[i] = distance.cosine(predicted_vectors[i],true_vectors[i])

  return distances


# accuracy
def ED_eval(eval_entity_labels,eval_ed_outputs,dataset_candidate_entities):
  # a list with predicted vectors
  predicted_vectors = []
  true_labels = []
  true_vectors = []
  locations= []
  #TODO all sets
  for i in range(len(eval_entity_labels)):
    for j in range(max_len):
      if(eval_entity_labels[i][j] != -100):
        true_labels.append(int_to_entity_map[eval_entity_labels[i][j]])
        true_vectors.append(entity_vector_dict[int_to_entity_map[eval_entity_labels[i][j]]])
        predicted_vectors.append(eval_ed_outputs[i][j].cpu().numpy())
        locations.append((i,j))
  # print(len(true_labels))
  entity_predictions = []
  # errors caused by entities not in candidate sets
  OOCS = 0
  for i in range(len(true_labels)):
    if(len(dataset_candidate_entities[i]) == 0):
      print("NO CANDIDATES CASE: ")
      entity_predictions.append("NOCANDIDATES")
      continue
    
    candidate_vectors = get_candidates_vectors(dataset_candidate_entities[i])
    sims = cosine_similarity(candidate_vectors,[predicted_vectors[i]])
    most_similar_candidate_index = np.argmax(sims)
    max_score = np.max(sims)
    most_similar_candidate = dataset_candidate_entities[i][most_similar_candidate_index]
    entity_predictions.append(most_similar_candidate)

    # DEBUG
    real_label_cos = cosine_similarity([true_vectors[i]],[predicted_vectors[i]])[0][0]
    # print(true_vectors[0])
    
    if(most_similar_candidate != true_labels[i] and (true_labels[i] not in dataset_candidate_entities[i]) and real_label_cos > max_score):
      print('DEBUG: REAL: {}, {}\tPREDICTED: {}, {} @{}'.format(true_labels[i],real_label_cos,most_similar_candidate,max_score,locations[i]))

      # print(locations[i])
      OOCS += 1
  print("error by missing candidates: " ,(OOCS/len(true_labels)))
  return true_labels,entity_predictions

In [ ]:
import pickle
N_CANDIDATES = 50

with open (f'drive/My Drive/train_candidate_entities{N_CANDIDATES}.pkl', 'rb') as fp:
    train_candidate_entities = pickle.load(fp)

with open (f'drive/My Drive/val_candidate_entities{N_CANDIDATES}.pkl', 'rb') as fp:
    val_candidate_entities = pickle.load(fp)

with open (f'drive/My Drive/test_candidate_entities{N_CANDIDATES}.pkl', 'rb') as fp:
    test_candidate_entities = pickle.load(fp)

In [ ]:
NEG_EXAMPLES = 10
NEG_EXAMPLE_LABEL = 0

global_counter = 0
for i in range(len(train_samples)):
  labels = []
  for gold_entity in train_samples[i].entities:
    span_labels = []
    # remove gold from the candidate set
    neg_examples = [(item[0],item[1],NEG_EXAMPLE_LABEL) for item in train_candidate_entities[global_counter] if item[0] != gold_entity]
    # keeping only the top neg examples
    neg_examples = neg_examples[:NEG_EXAMPLES]

    span_labels.append((gold_entity,1))

    span_labels.extend(neg_examples)

    global_counter+=1

    random.shuffle(span_labels)
    labels.append(span_labels)

  train_samples[i].add_ed_labels(labels)


global_counter = 0
NEG_EXAMPLES_TESTING = 10

for i in range(len(val_samples)):
  labels = []
  for gold_entity in val_samples[i].entities:
    span_labels = []
    # remove gold from the candidate set
    neg_examples = [(item[0],item[1],NEG_EXAMPLE_LABEL) for item in val_candidate_entities[global_counter] if item[0] != gold_entity]
    # keeping only the top neg examples
    neg_examples = neg_examples[:NEG_EXAMPLES_TESTING]

    span_labels.append((gold_entity,1))

    span_labels.extend(neg_examples)

    global_counter+=1
    random.shuffle(span_labels)

    labels.append(span_labels)

  val_samples[i].add_ed_labels(labels)


global_counter = 0
for i in range(len(test_samples)):
  labels = []
  for gold_entity in test_samples[i].entities:
    span_labels = []
    # remove gold from the candidate set
    neg_examples = [(item[0],item[1],NEG_EXAMPLE_LABEL) for item in test_candidate_entities[global_counter] if item[0] != gold_entity]
    # keeping only the top neg examples
    neg_examples = neg_examples[:NEG_EXAMPLES_TESTING]

    span_labels.append((gold_entity,1))

    span_labels.extend(neg_examples)
    random.shuffle(span_labels)

    global_counter+=1
    labels.append(span_labels)

  test_samples[i].add_ed_labels(labels)


In [ ]:
candidate_set = set()
for sample in train_samples:
  for span_labels in sample.ed_labels:
    candidate_set.update([item[0] for item in span_labels])
for sample in val_samples:
  for span_labels in sample.ed_labels:
    candidate_set.update([item[0] for item in span_labels])
for sample in test_samples:
  for span_labels in sample.ed_labels:
    candidate_set.update([item[0] for item in span_labels])

print("number of gold unique entities in all sets: ", len(candidate_set))

#use this to only keep the necessary one for this experiment 
entity_vector_dict,errors = get_needed_entity_vectors(ebert_emb,candidate_set)
print("number of entities found in trained wikipedia2vec: {}\nnumber of entities not found: {}".format(len(entity_vector_dict),len(errors)))

number of gold unique entities in all sets:  38845
number of entities found in trained wikipedia2vec: 38845
number of entities not found: 0


In [ ]:
# use this as label for ED
entity_to_int_map = dict()
for key,i in zip(entity_vector_dict, range(len(entity_vector_dict))):
  entity_to_int_map[key] = i


int_to_entity_map = {v:k for k,v in entity_to_int_map.items()}

In [ ]:
########### TRY candidate set and check for accuracy

In [ ]:
golden_ents = []
for sample in val_samples:
  golden_ents.extend(sample.entities)

In [ ]:
errors = 0
for i in range(len(golden_ents)):
  if golden_ents[i] not in [i[0] for i in val_candidate_entities[i]]:
    errors+=1

1.0- errors/len(val_candidate_entities) 

0.9629319371727749

In [ ]:
errors = 0
for i in range(len(golden_ents)):
  try:
    if not (golden_ents[i] == val_candidate_entities[i][0] or golden_ents[i] == val_candidate_entities[i][1] or golden_ents[i] == val_candidate_entities[i][2]):
      errors+=1
  except:
    errors+=1

errors

4775

In [ ]:
def get_entity_vectors_v2(batch_ed_labels,NEG_EXAMPLES=2):
  # entity_vectors = torch.zeros((len(batch_ed_labels),NEG_EXAMPLES,entity_dim),dtype=torch.float32)
  list_of_vectors = []
  list_of_labels = []
  counter = 0
  for i,sample in enumerate(batch_ed_labels):
    sample_vectors = []
    sample_labels = []
    # print(sample_labels)
    # print(len(sample))
    for span in sample:
      span_vectors= []
      span_labels = []
      # if(len(span) != 3):
      #   print(span)
      for example in span:
        counter+=1
        span_vectors.append(torch.tensor(entity_vector_dict[example[0]]))
        span_labels.append(example[1])

      sample_vectors.append(span_vectors)
      sample_labels.append(span_labels)

    list_of_vectors.append(sample_vectors)
    list_of_labels.append(sample_labels)

  return list_of_vectors,list_of_labels,counter


def get_entity_vectors_v3(batch_ed_labels,NEG_EXAMPLES=2,MISSING_PRIOR_VALUE = 1e-6,return_entity_labels=False):
  # entity_vectors = torch.zeros((len(batch_ed_labels),NEG_EXAMPLES,entity_dim),dtype=torch.float32)
  list_of_vectors = []
  list_of_labels = []
  list_of_priors = []
  list_of_entities=[]

  counter = 0
  for i,sample in enumerate(batch_ed_labels):
    sample_vectors = []
    sample_labels = []
    sample_priors = []
    sample_entities = []

    
    # print(sample_labels)
    # print(len(sample))
    for span in sample:
      span_vectors= []
      span_priors= []
      span_labels = []
      span_entities = []

      # if(len(span) != 3):
      #   print(span)
      for example in span:
        counter+=1
        span_entities.append(example[0])
        span_vectors.append(torch.tensor(entity_vector_dict[example[0]]))
        if(len(example) == 2):
          span_priors.append(torch.tensor(MISSING_PRIOR_VALUE))
          span_labels.append(example[1])
        else:
          span_priors.append(torch.tensor(example[1]))
          span_labels.append(example[2])

      sample_vectors.append(span_vectors)
      sample_priors.append(span_priors)
      sample_labels.append(span_labels)
      sample_entities.append(span_entities)

    list_of_vectors.append(sample_vectors)
    list_of_priors.append(sample_priors)
    list_of_labels.append(sample_labels)
    list_of_entities.append(sample_entities)

  if (return_entity_labels):
    return list_of_vectors,list_of_priors,list_of_labels,list_of_entities,counter
  return list_of_vectors,list_of_priors,list_of_labels,counter

# Joint Model

## NEW JOINT


In [ ]:
# wiki2vec entity dimensions #TODO try other sizes 
entity_dim = 768

hidden_dropout_prob = 0.1
bert_hidden_size = 768

# LEFT AND RIGHT
CONTEXT_SIZE = 10

bert = BertModel.from_pretrained('bert-base-cased',add_pooling_layer=False).cuda()

# hidden_layer = torch.nn.Linear(hidden_size, entity_dim).cuda()

# ed_dropout = torch.nn.Dropout(hidden_dropout_prob).cuda()
# ed_mapping_layer = torch.nn.Linear(bert_hidden_size, entity_dim).cuda()


# +1 for the mention averaged vector
context_layer_input_dim = bert_hidden_size*(CONTEXT_SIZE+CONTEXT_SIZE+1)
context_layer = torch.nn.Linear(context_layer_input_dim, entity_dim).cuda()

# TODO add prior and hyperlink form? 
# contextvec + entityvec+ dotProduct 
ranking_layer_input_dim = entity_dim + entity_dim + 1 + 1
ranking_layer = torch.nn.Linear(ranking_layer_input_dim, 1).cuda()

# IOB tags
num_labels = 3
ner_dropout = torch.nn.Dropout(hidden_dropout_prob).cuda()
ner_classifier = torch.nn.Linear(bert_hidden_size, num_labels).cuda()



Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
learning_rate = 2e-5
# params_to_optimize = list(ed_mapping_layer.parameters()) +list(bert.parameters()) + list(hidden_layer.parameters())
params_to_optimize = list(ner_classifier.parameters()) + list(bert.parameters()) + list(context_layer.parameters()) + list(ranking_layer.parameters())
# params_to_optimize = list(ed_mapping_layer.parameters()) +list(bert.parameters())

optimizer_ed = AdamW(params_to_optimize, lr=learning_rate)

relu = torch.nn.ReLU()
cross_ent = torch.nn.CrossEntropyLoss(reduction="sum").cuda()

loss_lambda = 0.2


In [ ]:
ranking_layer.load_state_dict(torch.load("drive/My Drive/ranking_layerJOINT43.hdf5"))
context_layer.load_state_dict(torch.load("drive/My Drive/context_layerJOINT43.hdf5"))
bert.load_state_dict(torch.load( "drive/My Drive/bertJOINT43.hdf5"))
ner_classifier.load_state_dict(torch.load( "drive/My Drive/ner_classifierJOINT43.hdf5"))



<All keys matched successfully>

In [ ]:
# torch.save(ner_classifier.state_dict(), "drive/My Drive/ner_classifierJOINT43.hdf5")
# torch.save(ranking_layer.state_dict(), "drive/My Drive/ranking_layerJOINT43.hdf5")
# torch.save(context_layer.state_dict(), "drive/My Drive/context_layerJOINT43.hdf5")
# torch.save(bert.state_dict(), "drive/My Drive/bertJOINT43.hdf5")


In [ ]:
batch_size = 4
Epochs = 4
# NEGATIVE_EXAMPLES_COUNT = 3

training_steps = int(len(train_samples)/batch_size)+1

for epoch in range(Epochs):

  random.shuffle(train_samples)
  print("Epoch ", epoch)
  # global_entity_counter = 0
  losses = []
  with tqdm(total=training_steps) as progress_bar:
    for i in range(0, len(train_samples), batch_size):
        bert.train()
        context_layer.train()
        ranking_layer.train()
        ner_classifier.train()
        # hidden_layer.train()

        batch_sentences = [sample.sentence for sample in train_samples[i:i+batch_size]]
        batch_ner_labels = [sample.iob_labels for sample in train_samples[i:i+batch_size]]
        batch_entities = [sample.entities for sample in train_samples[i:i+batch_size]]
        batch_ed = [sample.ed_labels for sample in train_samples[i:i+batch_size]]

        inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
        batch_ner_y = torch.LongTensor(inputs['labels']).cuda()
        batch_ed_label_y = torch.LongTensor(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map)).cuda()
        batch_ed_vector_y = get_entity_vectors(batch_ed_label_y,entity_vector_dict,int_to_entity_map).cuda()
        entity_locations,number_of_entities = get_entity_locations(batch_ner_y)

        # not all spans have negative samples
        # batch_ed_posneg_vector_x,batch_ed_posneg_y,num_of_examples = get_entity_vectors_v2(batch_ed)
        batch_ed_posneg_vector_x,batch_ed_posneg_prior_x,batch_ed_posneg_y,num_of_examples = get_entity_vectors_v3(batch_ed)

        



        outputs = bert(
              inputs['input_ids'].cuda(),
              attention_mask=inputs['attention_mask'].cuda(),
              token_type_ids=inputs['token_type_ids'].cuda()
          )



        ###### NER ###########


        sequence_output = ner_dropout(outputs[0])
        ner_logits = ner_classifier(sequence_output)

        loss = None
        if batch_ner_y is not None:
          

          # Only keep active parts of the loss
          if inputs['attention_mask'].cuda() is not None:
            active_loss = inputs['attention_mask'].cuda().view(-1) == 1
            active_logits = ner_logits.view(-1, num_labels)
            active_labels = torch.where(
                active_loss, batch_ner_y.view(-1), torch.tensor(cross_ent.ignore_index).type_as(batch_ner_y)
            )
            loss_ner = cross_ent(active_logits, active_labels)
          else:
            loss_ner = cross_ent(ner_logits.view(-1, num_labels), batch_ner_y.view(-1))

        


        ######### ED  ########

        counter = 0
        ED_outputs = torch.zeros((num_of_examples)).cuda()
        ED_labels = torch.zeros((num_of_examples)).cuda()
        for ex_i in range(len(batch_ed_vector_y)):
          for span_index,(sample_entity_location,span_size) in enumerate(entity_locations[ex_i]):

            context_start = sample_entity_location - CONTEXT_SIZE
            context_end = sample_entity_location + CONTEXT_SIZE

            # ensure a 2*CONTEXT_SIZE sequence
            if (context_start < 0):
              context_start = 0
              
              context_end = CONTEXT_SIZE+CONTEXT_SIZE
              
            # ensure a 2*CONTEXT_SIZE sequence
            if (context_end >= max_len):

              context_start = max_len-CONTEXT_SIZE-CONTEXT_SIZE
              context_end = max_len
              
            assert outputs[0][ex_i][context_start:context_end].shape[0] == 2*CONTEXT_SIZE

            context_vector = outputs[0][ex_i][context_start:context_end]

            #TODO
            if (span_size == 1):
              mention_vector = outputs[0][ex_i][sample_entity_location]

            # avg mention embeddings
            else:
              mention_vector = torch.mean(outputs[0][ex_i][sample_entity_location:(sample_entity_location+span_size)],dim=0)

            mention_vector = mention_vector.view(-1, entity_dim)
            concat_vec = torch.cat((mention_vector,context_vector)).view(-1)
            
            context_output = context_layer(concat_vec)


            ## loop for each pos neg example 
            ## should be NEG + 1 POS
            for span_example_index in range(len(batch_ed_posneg_vector_x[ex_i][span_index])):
              entity_vector = batch_ed_posneg_vector_x[ex_i][span_index][span_example_index].cuda()
              entity_label = torch.tensor(batch_ed_posneg_y[ex_i][span_index][span_example_index])
              entity_prior = batch_ed_posneg_prior_x[ex_i][span_index][span_example_index].cuda()

              dot_sim = torch.dot(context_output,entity_vector)

              combined_vec = torch.cat((context_output,entity_vector)).view(-1)
              combined_vec = torch.cat((combined_vec,dot_sim.view(1)))
              combined_vec = torch.cat((combined_vec,entity_prior.view(1)))

              ranking_out = ranking_layer(combined_vec)

              ED_outputs[counter] = ranking_out
              ED_labels[counter] = entity_label

              counter += 1

        
        loss1 = ED_labels * relu(0.2 - ED_outputs)
        loss2 = (1 - ED_labels) * relu(ED_outputs)
        loss_ed = loss1 + loss2
        loss_ed = torch.sum(loss_ed)



        loss = loss_lambda * loss_ner + (1-loss_lambda) * loss_ed



        # skip sequences with no entities (no loss)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(ner_classifier.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(context_layer.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(ranking_layer.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(bert.parameters(), 1.0)

        optimizer_ed.step()
        optimizer_ed.zero_grad()

        losses.append(loss.item())
        
        avg_loss = sum(losses)/len(losses)
        progress_bar.set_description("avg loss so far = {}".format(avg_loss))
        progress_bar.update(1)
        


    if(epoch % 3 == 0):
      eval_ed_outputs = []
      eval_entity_labels = []
      eval_sentences = []
      val_losses=[]
      ner_preds = []
      ner_eval_labels = []

      bert.eval()
      # ed_mapping_layer.eval()
      context_layer.eval()
      ranking_layer.eval()
      ner_classifier.eval()
      # ed_dropout.eval()
      # hidden_layer.eval()

      val_steps = int(len(val_samples)/batch_size)+1
      with tqdm(total=val_steps) as progress_bar:
        for i in range(0, len(val_samples), batch_size):

            with torch.no_grad():
              batch_sentences = [sample.sentence for sample in val_samples[i:i+batch_size]]
              batch_ner_labels = [sample.iob_labels for sample in val_samples[i:i+batch_size]]
              batch_entities = [sample.entities for sample in val_samples[i:i+batch_size]]
              batch_ed = [sample.ed_labels for sample in val_samples[i:i+batch_size]]

              inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
              batch_ner_y = torch.LongTensor(inputs['labels']).cuda()
              batch_ed_label_y = torch.LongTensor(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map)).cuda()
              batch_ed_vector_y = get_entity_vectors(batch_ed_label_y,entity_vector_dict,int_to_entity_map).cuda()
              entity_locations,number_of_entities = get_entity_locations(batch_ner_y)
              # batch_ed_posneg_vector_x,batch_ed_posneg_y,num_of_examples = get_entity_vectors_v2(batch_ed)
              batch_ed_posneg_vector_x,batch_ed_posneg_prior_x,batch_ed_posneg_y,num_of_examples = get_entity_vectors_v3(batch_ed)



              outputs = bert(
                    inputs['input_ids'].cuda(),
                    attention_mask=inputs['attention_mask'].cuda(),
                    token_type_ids=inputs['token_type_ids'].cuda()
                )
              
              sequence_output = outputs[0]
              ner_logits = ner_classifier(sequence_output)
              ner_eval_labels.extend(batch_ner_y)
              ner_preds.extend(torch.argmax(ner_logits.softmax(dim = 1),axis=2))




              counter = 0
              # ED_outputs = torch.zeros((num_of_examples)).cuda()
              ED_outputs = []
              ED_labels = []
              # ED_labels = torch.zeros((num_of_examples)).cuda()
              for ex_i in range(len(batch_ed_vector_y)):
                for span_index,(sample_entity_location,span_size) in enumerate(entity_locations[ex_i]):

                  context_start = sample_entity_location - CONTEXT_SIZE
                  context_end = sample_entity_location + CONTEXT_SIZE

                  # ensure a 2*CONTEXT_SIZE sequence
                  if (context_start < 0):
                    context_start = 0
                    
                    context_end = CONTEXT_SIZE+CONTEXT_SIZE
                    
                  # ensure a 2*CONTEXT_SIZE sequence
                  if (context_end >= max_len):

                    context_start = max_len-CONTEXT_SIZE-CONTEXT_SIZE
                    context_end = max_len
                    
                  assert outputs[0][ex_i][context_start:context_end].shape[0] == 2*CONTEXT_SIZE

                  context_vector = outputs[0][ex_i][context_start:context_end]

                  #TODO
                  if (span_size == 1):
                    mention_vector = outputs[0][ex_i][sample_entity_location]

                  # avg mention embeddings
                  else:
                    mention_vector = torch.mean(outputs[0][ex_i][sample_entity_location:(sample_entity_location+span_size)],dim=0)

                  mention_vector = mention_vector.view(-1, entity_dim)
                  concat_vec = torch.cat((mention_vector,context_vector)).view(-1)
                  
                  context_output = context_layer(concat_vec)

                  ## loop for each pos neg example 
                  ## should be NEG + 1 POS
                  span_outputs = []
                  span_labels = []
                  for span_example_index in range(len(batch_ed_posneg_vector_x[ex_i][span_index])):
                    entity_vector = batch_ed_posneg_vector_x[ex_i][span_index][span_example_index].cuda()
                    entity_label = torch.tensor(batch_ed_posneg_y[ex_i][span_index][span_example_index])
                    entity_prior = batch_ed_posneg_prior_x[ex_i][span_index][span_example_index].cuda()


                    dot_sim = torch.dot(context_output,entity_vector)

                    combined_vec = torch.cat((context_output,entity_vector)).view(-1)
                    combined_vec = torch.cat((combined_vec,dot_sim.view(1)))
                    combined_vec = torch.cat((combined_vec,entity_prior.view(1)))


                    ranking_out = ranking_layer(combined_vec)

                    span_outputs.append(ranking_out.item())
                    span_labels.append(entity_label.item())
                    # ED_outputs[counter] = ranking_out
                    # ED_labels[counter] = entity_label

                    counter += 1
                  ED_outputs.append(span_outputs)
                  ED_labels.append(span_labels)
              # loss1 = ED_labels * relu(0.2 - ED_outputs)
              # loss2 = (1 - ED_labels) * relu(ED_outputs)
              # val_loss = loss1 + loss2
              # val_losses.append(torch.sum(val_loss).item())


              progress_bar.update(1)
              eval_ed_outputs.extend(ED_outputs)
              eval_entity_labels.extend(ED_labels)
      eval_preds(ner_eval_labels,ner_preds)
      errors = 0
      for span_outputs,span_labels in zip(eval_ed_outputs,eval_entity_labels):
        if np.argmax(np.array(span_outputs)) != np.argmax(np.array(span_labels)):
          errors +=1
      print("ED acc:",(1-(errors/len(eval_ed_outputs))))
                    
      # print("val loss",sum(val_losses))

  0%|          | 0/274 [00:00<?, ?it/s]

Epoch  0


 99%|█████████▊| 68/69 [00:40<00:00,  1.67it/s]



0.8982483763038772


  0%|          | 0/274 [00:00<?, ?it/s]

...

Epoch  3


 99%|█████████▊| 68/69 [00:40<00:00,  1.68it/s]



0.9057131442269277


avg loss so far = 0.9396917732682311: 100%|██████████| 274/274 [05:18<00:00,  1.16s/it]

              precision    recall  f1-score   support

           _       0.87      0.95      0.91      4775

   micro avg       0.87      0.95      0.91      4775
   macro avg       0.87      0.95      0.91      4775
weighted avg       0.87      0.95      0.91      4775


ED acc: 0.9124607329842932


In [ ]:
batch_size = 4

eval_ed_outputs = []
eval_entity_labels = []
eval_sentences = []
val_losses=[]
ner_preds = []
ner_eval_labels = []
eval_entity_names=[]

bert.eval()
# ed_mapping_layer.eval()
context_layer.eval()
ranking_layer.eval()
ner_classifier.eval()
# ed_dropout.eval()
# hidden_layer.eval()

val_steps = int(len(test_samples)/batch_size)+1
with tqdm(total=val_steps) as progress_bar:
  for i in range(0, len(test_samples), batch_size):

      with torch.no_grad():
        batch_sentences = [sample.sentence for sample in test_samples[i:i+batch_size]]
        batch_ner_labels = [sample.iob_labels for sample in test_samples[i:i+batch_size]]
        batch_entities = [sample.entities for sample in test_samples[i:i+batch_size]]
        batch_ed = [sample.ed_labels for sample in test_samples[i:i+batch_size]]

        inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
        batch_ner_y = torch.LongTensor(inputs['labels']).cuda()
        batch_ed_label_y = torch.LongTensor(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map)).cuda()
        batch_ed_vector_y = get_entity_vectors(batch_ed_label_y,entity_vector_dict,int_to_entity_map).cuda()
        entity_locations,number_of_entities = get_entity_locations(batch_ner_y)
        # batch_ed_posneg_vector_x,batch_ed_posneg_y,num_of_examples = get_entity_vectors_v2(batch_ed)
        batch_ed_posneg_vector_x,batch_ed_posneg_prior_x,batch_ed_posneg_y,batch_ed_entity_names,num_of_examples = get_entity_vectors_v3(batch_ed,return_entity_labels=True)



        outputs = bert(
              inputs['input_ids'].cuda(),
              attention_mask=inputs['attention_mask'].cuda(),
              token_type_ids=inputs['token_type_ids'].cuda()
          )
        
        sequence_output = outputs[0]
        ner_logits = ner_classifier(sequence_output)
        ner_eval_labels.extend(batch_ner_y)
        ner_preds.extend(torch.argmax(ner_logits.softmax(dim = 1),axis=2))




        counter = 0
        # ED_outputs = torch.zeros((num_of_examples)).cuda()
        ED_outputs = []
        ED_labels = []
        ED_entity_names = []
        # ED_labels = torch.zeros((num_of_examples)).cuda()
        for ex_i in range(len(batch_ed_vector_y)):
          for span_index,(sample_entity_location,span_size) in enumerate(entity_locations[ex_i]):

            context_start = sample_entity_location - CONTEXT_SIZE
            context_end = sample_entity_location + CONTEXT_SIZE

            # ensure a 2*CONTEXT_SIZE sequence
            if (context_start < 0):
              context_start = 0
              
              context_end = CONTEXT_SIZE+CONTEXT_SIZE
              
            # ensure a 2*CONTEXT_SIZE sequence
            if (context_end >= max_len):

              context_start = max_len-CONTEXT_SIZE-CONTEXT_SIZE
              context_end = max_len
              
            assert outputs[0][ex_i][context_start:context_end].shape[0] == 2*CONTEXT_SIZE

            context_vector = outputs[0][ex_i][context_start:context_end]

            #TODO
            if (span_size == 1):
              mention_vector = outputs[0][ex_i][sample_entity_location]

            # avg mention embeddings
            else:
              mention_vector = torch.mean(outputs[0][ex_i][sample_entity_location:(sample_entity_location+span_size)],dim=0)

            mention_vector = mention_vector.view(-1, entity_dim)
            concat_vec = torch.cat((mention_vector,context_vector)).view(-1)
            
            context_output = context_layer(concat_vec)

            ## loop for each pos neg example 
            ## should be NEG + 1 POS
            span_outputs = []
            span_labels = []
            span_entitiy_names = []
            for span_example_index in range(len(batch_ed_posneg_vector_x[ex_i][span_index])):
              entity_vector = batch_ed_posneg_vector_x[ex_i][span_index][span_example_index].cuda()
              entity_label = torch.tensor(batch_ed_posneg_y[ex_i][span_index][span_example_index])
              entity_prior = batch_ed_posneg_prior_x[ex_i][span_index][span_example_index].cuda()

              


              dot_sim = torch.dot(context_output,entity_vector)

              combined_vec = torch.cat((context_output,entity_vector)).view(-1)
              combined_vec = torch.cat((combined_vec,dot_sim.view(1)))
              combined_vec = torch.cat((combined_vec,entity_prior.view(1)))


              ranking_out = ranking_layer(combined_vec)

              span_outputs.append(ranking_out.item())
              span_labels.append(entity_label.item())


              entity_name = batch_ed_entity_names[ex_i][span_index][span_example_index]
              span_entitiy_names.append(entity_name)

              # ED_outputs[counter] = ranking_out
              # ED_labels[counter] = entity_label

              counter += 1
            ED_outputs.append(span_outputs)
            ED_labels.append(span_labels)
            ED_entity_names.append(span_entitiy_names)
        # loss1 = ED_labels * relu(0.2 - ED_outputs)
        # loss2 = (1 - ED_labels) * relu(ED_outputs)
        # val_loss = loss1 + loss2
        # val_losses.append(torch.sum(val_loss).item())


        progress_bar.update(1)
        eval_ed_outputs.extend(ED_outputs)
        eval_entity_labels.extend(ED_labels)
        eval_entity_names.extend(ED_entity_names)
eval_preds(ner_eval_labels,ner_preds)
errors = 0
for span_outputs,span_labels in zip(eval_ed_outputs,eval_entity_labels):
  if np.argmax(np.array(span_outputs)) != np.argmax(np.array(span_labels)):
    errors +=1
print("ED acc:",(1-(errors/len(eval_ed_outputs))))

100%|██████████| 67/67 [01:08<00:00,  1.02s/it]



0.8608150470219437
              precision    recall  f1-score   support

           _       0.81      0.92      0.86      4477

   micro avg       0.81      0.92      0.86      4477
   macro avg       0.81      0.92      0.86      4477
weighted avg       0.81      0.92      0.86      4477


ED acc: 0.8588340406522225


In [ ]:
ed_gold = []
ed_predicted = []
for span_outputs,span_labels,span_names in zip(eval_ed_outputs,eval_entity_labels,eval_entity_names):
  prediction = span_names[np.argmax(np.array(span_outputs))]
  golden_entity = span_names[np.argmax(np.array(span_labels))]
  ed_gold.append(golden_entity)
  ed_predicted.append(prediction)

In [ ]:
true_predictions = [
    [int_to_label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(ner_preds, ner_eval_labels)
]
true_labels = [
    [int_to_label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(ner_preds, ner_eval_labels)
]

In [ ]:
counter = -1
e2e_gold = []
e2e_perfectNER_ed_preds = []

for sample in true_labels:
  sample_gold = []
  sample_perfectNER_preds = []
  for label in sample:
    if (label == "B"):
      counter += 1
      sample_gold.append("B-"+str(entity_to_int_map[ed_gold[counter]]))
      sample_perfectNER_preds.append("B-"+str(entity_to_int_map[ed_predicted[counter]]))

    elif (label == "I"):
      sample_gold.append("I-"+str(entity_to_int_map[ed_gold[counter]]))
      sample_perfectNER_preds.append("I-"+str(entity_to_int_map[ed_predicted[counter]]))

    elif (label == "O"):
      sample_gold.append("O")
      sample_perfectNER_preds.append("O")
  
  e2e_gold.append(sample_gold)
  e2e_perfectNER_ed_preds.append(sample_perfectNER_preds)


In [ ]:
final_e2e_preds = []
for sample,pred_sample,ed_sample_pred in zip(true_labels,true_predictions,e2e_perfectNER_ed_preds):
  sample_e2e_preds = []
  for gold,pred,ed_pred in zip(sample,pred_sample,ed_sample_pred):
    if(pred == gold):
      sample_e2e_preds.append(ed_pred)
    else:
      sample_e2e_preds.append(pred)
  final_e2e_preds.append(sample_e2e_preds)


In [ ]:
from seqeval.metrics import classification_report
print(classification_report(e2e_gold,final_e2e_preds,digits=4))

/usr/local/lib/python3.6/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       10001     0.0000    0.0000    0.0000         4
       10019     1.0000    1.0000    1.0000         1
       10023     1.0000    1.0000    1.0000         1
       10046     1.0000    1.0000    1.0000         1
       10099     1.0000    1.0000    1.0000         1
       10116     0.0000    0.0000    0.0000         2
       10121     0.0000    0.0000    0.0000         1
       10123     0.0000    0.0000    0.0000         0
       10135     0.0000    0.0000    0.0000         0
       10185     1.0000    1.0000    1.0000         1
       10201     1.0000    0.3333    0.5000         3
       10210     0.0000    0.0000    0.0000         0
       10213     1.0000    1.0000    1.0000         1
       10214     0.0000    0.0000    0.0000         2
       10225     1.0000    1.0000    1.0000         4
       10268     1.0000    1.0000    1.0000         2
       10271     0.0000    0.0000    0.0000         1
       10330     1.0000    

In [ ]:
e2e_gold[0]

In [ ]:
accuracy_score(e2e_gold,final_e2e_preds)

0.9906591166922861

## OLD JOINT

In [ ]:
hidden_dropout_prob = 0.1
hidden_size = 768

bert = BertModel.from_pretrained('bert-base-cased',add_pooling_layer=False).cuda()

# wiki2vec entity dimensions #TODO try other sizes 
entity_dim = 768
ed_dropout = torch.nn.Dropout(hidden_dropout_prob).cuda()
ed_mapping_layer = torch.nn.Linear(hidden_size, entity_dim).cuda()

# IOB tags
num_labels = 3
ner_dropout = torch.nn.Dropout(hidden_dropout_prob).cuda()
ner_classifier = torch.nn.Linear(hidden_size, num_labels).cuda()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
learning_rate = 2e-5

params_to_optimize = list(ner_classifier.parameters()) + list(ed_mapping_layer.parameters()) + list(bert.parameters())
optimizer = AdamW(params_to_optimize, lr=learning_rate)

# cos_sim = torch.nn.CosineSimilarity()
cross_ent = torch.nn.CrossEntropyLoss().cuda()

loss_lambda = 0.5

In [ ]:
batch_size = 4
Epochs = 4


training_steps = int(len(train_samples)/batch_size)+1
losses = []
for epoch in range(Epochs):
  with tqdm(total=training_steps) as progress_bar:
    for i in range(0, len(train_samples), batch_size):

        bert.train()
        ner_classifier.train()
        ner_dropout.train()
        ed_mapping_layer.train()
        ed_dropout.train()

        batch_sentences = [sample.sentence for sample in train_samples[i:i+batch_size]]
        batch_sentences = [sample.sentence for sample in train_samples[i:i+batch_size]]
        batch_ner_labels = [sample.iob_labels for sample in train_samples[i:i+batch_size]]
        batch_entities = [sample.entities for sample in train_samples[i:i+batch_size]]

        inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
        batch_ner_y = torch.LongTensor(inputs['labels']).cuda()
        batch_ed_label_y = torch.LongTensor(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map)).cuda()
        batch_ed_vector_y = get_entity_vectors(batch_ed_label_y,entity_vector_dict,int_to_entity_map).cuda()
        entity_locations,number_of_entities = get_entity_locations(batch_ed_label_y)

        outputs = bert(
              inputs['input_ids'].cuda(),
              attention_mask=inputs['attention_mask'].cuda(),
              token_type_ids=inputs['token_type_ids'].cuda()
          )

        ##### NER ####
        sequence_output = ner_dropout(outputs[0])
        ner_logits = ner_classifier(sequence_output)

        loss = None
        if batch_ner_y is not None:
          

          # Only keep active parts of the loss
          if inputs['attention_mask'].cuda() is not None:
            active_loss = inputs['attention_mask'].cuda().view(-1) == 1
            active_logits = ner_logits.view(-1, num_labels)
            active_labels = torch.where(
                active_loss, batch_ner_y.view(-1), torch.tensor(cross_ent.ignore_index).type_as(batch_ner_y)
            )
            loss_ner = cross_ent(active_logits, active_labels)
          else:
            loss_ner = cross_ent(ner_logits.view(-1, num_labels), batch_ner_y.view(-1))



        #### ED ####
        
        sequence_output = ed_dropout(outputs[0])

        ed_logits = ed_mapping_layer(sequence_output)

        single_losses = torch.zeros(number_of_entities).cuda()
        counter = 0
        for ex_i in range(len(batch_ed_vector_y)):
          for sample_entity_location in entity_locations[ex_i]:
            single_loss = 1 - torch.abs(cos_sim(ed_logits[ex_i][sample_entity_location].view(-1,entity_dim),batch_ed_vector_y[ex_i][sample_entity_location].view(-1,entity_dim)))
            single_losses[counter] = single_loss
            counter += 1
        loss_ed = torch.mean(single_losses)
        
        # J(θ) = λ Lmd(θ) + (1 − λ) Led(θ)
        loss = loss_lambda * loss_ner + (1-loss_lambda) * loss_ed

        # TODO should I clip bert aswell?   
        torch.nn.utils.clip_grad_norm_(ed_mapping_layer.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(ner_classifier.parameters(), 1.0)
        # torch.nn.utils.clip_grad_norm_(bert.parameters(), 1.0)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        losses.append(loss.item())
        avg_loss = sum(losses)/len(losses)
        progress_bar.update(1)
        progress_bar.set_description("avg loss so far = {}".format(avg_loss))


    if(epoch % 3 == 0):
      eval_ed_outputs = []
      eval_entity_labels = []
      # TODO as function
      bert.eval()
      ner_classifier.eval()
      ner_dropout.eval()
      ed_mapping_layer.eval()
      ed_dropout.eval()

      ner_preds = []
      ner_eval_labels = []

      val_steps = int(len(val_samples)/batch_size)+1

      
      for i in range(0, len(val_samples), batch_size):
        with torch.no_grad():
          batch_sentences = [sample.sentence for sample in val_samples[i:i+batch_size]]
          batch_sentences = [sample.sentence for sample in val_samples[i:i+batch_size]]
          batch_ner_labels = [sample.iob_labels for sample in val_samples[i:i+batch_size]]
          batch_entities = [sample.entities for sample in val_samples[i:i+batch_size]]

          inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
          batch_ner_y = torch.LongTensor(inputs['labels']).cuda()
          batch_ed_label_y = torch.LongTensor(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map)).cuda()
          batch_ed_vector_y = get_entity_vectors(batch_ed_label_y,entity_vector_dict,int_to_entity_map).cuda()

          outputs = bert(
                inputs['input_ids'].cuda(),
                attention_mask=inputs['attention_mask'].cuda(),
                token_type_ids=inputs['token_type_ids'].cuda()
            )
          
          sequence_output = outputs[0]
          ner_logits = ner_classifier(sequence_output)
          ner_eval_labels.extend(batch_ner_y)
          ner_preds.extend(torch.argmax(ner_logits.softmax(dim = 1),axis=2))

          sequence_output = ed_dropout(outputs[0])
          ed_logits = ed_mapping_layer(sequence_output)
          eval_ed_outputs.extend(ed_logits)
          eval_entity_labels.extend(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map))
              
      eval_preds(ner_eval_labels,ner_preds)
      distances = ED_avg_cos_dis(eval_entity_labels,eval_ed_outputs)
      print('Distances: ',np.mean(distances))

avg loss so far = 0.051680085473578344: 100%|██████████| 274/274 [02:06<00:00,  2.28it/s]


0.899549637752105
              precision    recall  f1-score   support

           _       0.84      0.96      0.90      4775

   micro avg       0.84      0.96      0.90      4775
   macro avg       0.84      0.96      0.90      4775
weighted avg       0.84      0.96      0.90      4775




  0%|          | 0/274 [00:00<?, ?it/s]

Distances:  0.16588060278133415


avg loss so far = 0.05134946413603306: 100%|██████████| 274/274 [02:08<00:00,  2.23it/s]


0.8996297018125122
              precision    recall  f1-score   support

           _       0.84      0.97      0.90      4775

   micro avg       0.84      0.97      0.90      4775
   macro avg       0.84      0.97      0.90      4775
weighted avg       0.84      0.97      0.90      4775




avg loss so far = 0.05134946413603306: 100%|██████████| 274/274 [02:32<00:00,  1.80it/s]

Distances:  0.16416379881544887


## save model

In [ ]:
# !mkdir "drive/My Drive/e2e_trained_model" 
torch.save(ed_mapping_layer.state_dict(), "drive/My Drive/e2e_trained_model/ed_layer80.hdf5")
torch.save(bert.state_dict(), "drive/My Drive/e2e_trained_model/bert80.hdf5")
torch.save(ner_classifier.state_dict(), "drive/My Drive/e2e_trained_model/ner_layer80.hdf5")

## load model

In [ ]:
ner_classifier.load_state_dict(torch.load('drive/My Drive/e2e_trained_model/ner_layer60.hdf5'))
ed_mapping_layer.load_state_dict(torch.load('drive/My Drive/e2e_trained_model/ed_layer60.hdf5'))
bert.load_state_dict(torch.load('drive/My Drive/e2e_trained_model/bert60.hdf5'))


<All keys matched successfully>

## eval Seperatly (ED and NER performances)

In [ ]:
eval_ed_outputs = []
eval_entity_labels = []
# TODO as function
bert.eval()
ner_classifier.eval()
ner_dropout.eval()
ed_mapping_layer.eval()
ed_dropout.eval()

ner_preds = []
ner_eval_labels = []
batch_size = 8
dataset_samples = test_samples
val_steps = int(len(dataset_samples)/batch_size)+1


for i in range(0, len(dataset_samples), batch_size):
  with torch.no_grad():
    batch_sentences = [sample.sentence for sample in dataset_samples[i:i+batch_size]]
    batch_sentences = [sample.sentence for sample in dataset_samples[i:i+batch_size]]
    batch_ner_labels = [sample.iob_labels for sample in dataset_samples[i:i+batch_size]]
    batch_entities = [sample.entities for sample in dataset_samples[i:i+batch_size]]

    inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
    batch_ner_y = torch.LongTensor(inputs['labels']).cuda()
    batch_ed_label_y = torch.LongTensor(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map)).cuda()
    batch_ed_vector_y = get_entity_vectors(batch_ed_label_y,entity_vector_dict,int_to_entity_map).cuda()

    outputs = bert(
          inputs['input_ids'].cuda(),
          attention_mask=inputs['attention_mask'].cuda(),
          token_type_ids=inputs['token_type_ids'].cuda()
      )
    
    sequence_output = outputs[0]
    ner_logits = ner_classifier(sequence_output)
    ner_eval_labels.extend(batch_ner_y)
    ner_preds.extend(torch.argmax(ner_logits.softmax(dim = 1),axis=2))

    sequence_output = ed_dropout(outputs[0])
    ed_logits = ed_mapping_layer(sequence_output)
    eval_ed_outputs.extend(ed_logits)
    eval_entity_labels.extend(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map))
        
eval_preds(ner_eval_labels,ner_preds)
distances = ED_avg_cos_dis(eval_entity_labels,eval_ed_outputs)
print('Distances: ',np.mean(distances))


0.8606874110229815
              precision    recall  f1-score   support

           _       0.79      0.95      0.86      4477

   micro avg       0.79      0.95      0.86      4477
   macro avg       0.79      0.95      0.86      4477
weighted avg       0.79      0.95      0.86      4477


Distances:  0.22537053705779522


In [ ]:
import pickle

with open ('drive/My Drive/train_candidate_entities10.pkl', 'rb') as fp:
    train_candidate_entities = pickle.load(fp)

with open ('drive/My Drive/val_candidate_entities10.pkl', 'rb') as fp:
    val_candidate_entities = pickle.load(fp)

with open ('drive/My Drive/test_candidate_entities10.pkl', 'rb') as fp:
    test_candidate_entities = pickle.load(fp)

In [ ]:
true_labels,entity_predictions = ED_eval(eval_entity_labels,eval_ed_outputs,test_candidate_entities)
accuracy_score(true_labels,entity_predictions)

0.6924279651552379

## eval End2End Entity Linking

In [ ]:

bert.eval()
ner_classifier.eval()
ner_dropout.eval()
ed_mapping_layer.eval()
ed_dropout.eval()

eval_ed_outputs = []
eval_entity_labels = []
ner_preds = []
ner_eval_labels = []
tokenized_inputs = []

batch_size = 8
dataset_samples = test_samples
val_steps = int(len(dataset_samples)/batch_size)+1


for i in range(0, len(dataset_samples), batch_size):
  with torch.no_grad():
    batch_sentences = [sample.sentence for sample in dataset_samples[i:i+batch_size]]
    batch_sentences = [sample.sentence for sample in dataset_samples[i:i+batch_size]]
    batch_ner_labels = [sample.iob_labels for sample in dataset_samples[i:i+batch_size]]
    batch_entities = [sample.entities for sample in dataset_samples[i:i+batch_size]]

    inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
    batch_ner_y = torch.LongTensor(inputs['labels']).cuda()
    batch_ed_label_y = torch.LongTensor(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map)).cuda()
    batch_ed_vector_y = get_entity_vectors(batch_ed_label_y,entity_vector_dict,int_to_entity_map).cuda()

    outputs = bert(
          inputs['input_ids'].cuda(),
          attention_mask=inputs['attention_mask'].cuda(),
          token_type_ids=inputs['token_type_ids'].cuda()
      )
    
    sequence_output = outputs[0]
    ner_logits = ner_classifier(sequence_output)
    ner_eval_labels.extend(batch_ner_y)
    ner_preds.extend(torch.argmax(ner_logits.softmax(dim = 1),axis=2))

    sequence_output = ed_dropout(outputs[0])
    ed_logits = ed_mapping_layer(sequence_output)
    eval_ed_outputs.extend(ed_logits)
    eval_entity_labels.extend(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map))
    
    tokenized_inputs.extend(inputs['input_ids'])

In [ ]:
samples = test_samples

predicted_ner_labels = []
real_ner_labels = []
predicted_vectors = []
reconstructed_sentences = []
predicted_spans = []


for i in tqdm(range(len(samples))):
  real_ent_counter = -1

  real_sample_ner_labels = []
  predicted_sample_ner_labels = []

  tokenized_WP_sentence = tokenizer.convert_ids_to_tokens(tokenized_inputs[i])
  tokenized_clean_sentence = []
  
  predicted_sample_vectors = []

  predicted_sample_spans = []

  current_span_start = None
  current_span_end = None
  last_valid_index = None
  for j in range(max_len):
    # we need to skip -100 which are wordpeice non-head ##tokens
    if (tokenized_WP_sentence[j] != "[PAD]" ):
      last_valid_index = j
    if (ner_eval_labels[i][j] != -100):
      predicted_ner_label = ner_preds[i][j].item()
      predicted_sample_ner_labels.append(predicted_ner_label)

      
      # start of a new mention and keep the vector
      if (predicted_ner_label == 1):

        # keep the vector for B case which represents the whole mention
        predicted_sample_vectors.append(eval_ed_outputs[i][j].cpu().numpy())
        
        # append the previous span if any
        if (current_span_start != None):
          # consquetive mentions 
          if (current_span_end == None):
            # end is the previous token
            current_span_end = j
          predicted_sample_spans.append(tokenizer.convert_tokens_to_string(tokenized_WP_sentence[current_span_start:current_span_end]))

        # start new span
        current_span_start = j
        current_span_end = None


      # # continue the current span
      # elif (predicted_ner_label == 2):
      #   current_span_mid = j

      # end and append the current span
      elif (predicted_ner_label == 0):
        # append the previous span if any
        if (current_span_start != None):
          # consquetive mentions 
          if (current_span_end == None):
            # end is the previous token
            current_span_end = j
          predicted_sample_spans.append(tokenizer.convert_tokens_to_string(tokenized_WP_sentence[current_span_start:current_span_end]))

        # no new spans so far
        current_span_start = None
        current_span_end = None


    # B label
    if (ner_eval_labels[i][j] == 1):
      real_ent_counter += 1
      real_sample_ner_labels.append(samples[i].entities[real_ent_counter])

    # I label
    elif (ner_eval_labels[i][j] == 2):
      real_sample_ner_labels.append(samples[i].entities[real_ent_counter])
    
    # O label
    elif (ner_eval_labels[i][j] == 0):
      real_sample_ner_labels.append('O')

  # some word at the end
  if(current_span_start != None): 
    print("Word At the end case ", (i,last_valid_index))
    if (current_span_start == last_valid_index):
      predicted_sample_spans.append(tokenizer.convert_tokens_to_string(tokenized_WP_sentence[current_span_start]))
    else:
      predicted_sample_spans.append(tokenizer.convert_tokens_to_string(tokenized_WP_sentence[current_span_start:last_valid_index]))

  predicted_vectors.append(predicted_sample_vectors)
  real_ner_labels.append(real_sample_ner_labels)
  predicted_ner_labels.append(predicted_sample_ner_labels)
  predicted_spans.append(predicted_sample_spans)

 13%|█▎        | 35/265 [00:03<00:20, 11.41it/s]

Word At the end case  (33, 481)


 25%|██▌       | 67/265 [00:05<00:17, 11.45it/s]

Word At the end case  (64, 65)


 79%|███████▉  | 209/265 [00:18<00:04, 11.44it/s]

...

Word At the end case  (252, 430)


 98%|█████████▊| 259/265 [00:23<00:00, 11.11it/s]

Word At the end case  (257, 327)
Word At the end case  (259, 315)


100%|██████████| 265/265 [00:23<00:00, 11.18it/s]

Word At the end case  (263, 245)


In [ ]:
def get_prediction_candidates(predicted_spans,MAX_CANDIDATES):
  dataset_candidates = []

  for sample_spans in predicted_spans:
    sample_candidates = []
    for span in sample_spans:

      generator_output = candidate_generator.get_mentions_raw_text(span)

      candidates = []
      for i in range(len(generator_output['candidate_spans'])):
        candidates.extend(zip(generator_output['candidate_entities'][i],generator_output['candidate_entity_priors'][i]))
      candidates = sorted(candidates, key=lambda x: x[1],reverse=True)

      titles_only = [x[0] for x in candidates]
      filtered = filter_non_available_entities(titles_only)
      span_candidates = list(OrderedDict.fromkeys(filtered))
      # print(titles_only)
      if(len(span_candidates) > MAX_CANDIDATES ):
        span_candidates = span_candidates[:MAX_CANDIDATES]
      if(len(span_candidates) == 0):
        print('No candidates for: ', span)

      sample_candidates.append(span_candidates)
    dataset_candidates.append(sample_candidates)
  return dataset_candidates

In [ ]:
prediction_candidates = get_prediction_candidates(predicted_spans,MAX_CANDIDATES = 10)
entity_predictions = []
for i, sample_candidates in enumerate(prediction_candidates):
  sample_entities_preds = []
  for k, entity_candidates in enumerate(sample_candidates):
    if (len(entity_candidates) == 0):
      # couldn't find in candidateGenerator
      sample_entities_preds.append("NONFOUND")
      continue
    else:
      candidate_vectors = get_candidates_vectors(entity_candidates)
      sims = cosine_similarity(candidate_vectors,[predicted_vectors[i][k]])
      most_similar_candidate_index = np.argmax(sims)
      max_score = np.max(sims)
      most_similar_candidate = entity_candidates[most_similar_candidate_index]
      sample_entities_preds.append(most_similar_candidate)
  entity_predictions.append(sample_entities_preds)

No candidates for:  Giambatista Croci
No candidates for:  Engllsh
No candidates for:  Iordanescu
No candidates for:  Sodigraf
No candidates for:  Bormirow
No candidates for:  JosepGuardiola
No candidates for:  AbelardoFernandez
No candidates for:  Logrones
No candidates for:  Kovanda
No candidates for:  Mesipia
No candidates for:  Bajaia
No candidates for:  Piniel
No candidates for:  SAfrican
No candidates for:  Gaulieder
No candidates for:  Gaulieder
No candidates for:  Simec
No candidates for:  Camaras
No candidates for:  Sunseeds
No candidates for:  Qinglan
No candidates for:  Ranyon
No candidates for:  Lawhon
No candidates for:  Lawhon
No candidates for:  Voeks
No candidates for:  Voeks
No candidates for:  Voeks
No candidates for:  OPECNA
No candidates for:  of
No candidates for:  Kesers
No candidates for:  Karabuluts
No candidates for:  Zaireans
No candidates for:  Benchaou
No candidates for:  Benchaou
No candidates for:  Nukaya
No candidates for:  Innisbrook
No candidates for:  G

In [ ]:
final_predicted_ner_labels = []



for i in tqdm(range(len(samples))):
  ent_counter = -1


  final_predicted_sample_ner_labels = []

  for j in range((len(predicted_ner_labels[i]))):
    # we need to skip -100 which are wordpeice non-head ##tokens

    # B label
    if (predicted_ner_labels[i][j] == 1):
      ent_counter += 1
      final_predicted_sample_ner_labels.append(entity_predictions[i][ent_counter])

    # I label
    elif (predicted_ner_labels[i][j] == 2):
      final_predicted_sample_ner_labels.append(entity_predictions[i][ent_counter])
    
    # O label
    elif (predicted_ner_labels[i][j] == 0):
      final_predicted_sample_ner_labels.append('O')

    
  final_predicted_ner_labels.append(final_predicted_sample_ner_labels)

100%|██████████| 265/265 [00:00<00:00, 14357.38it/s]


In [ ]:
from seqeval.metrics import classification_report
print(classification_report(real_ner_labels,final_predicted_ner_labels,digits=4))

In [ ]:
# accuracy
def E2E_EL_eval(ner_eval_labels,ner_preds,eval_entity_labels,eval_ed_outputs,candidate_generator):
  predicted_ner_labels = []
  real_ner_labels = []
  for i in range(len(eval_entity_labels)):
    last_entity = None
    for j in range(max_len):
      if (ner_eval_labels[i][j] == 1):
        ner_eval_labels
        

  # a list with predicted vectors
  predicted_vectors = []
  true_labels = []
  true_vectors = []
  for i in range(len(eval_entity_labels)):
    for j in range(max_len):
      if(eval_entity_labels[i][j] != -100):
        


        true_labels.append(int_to_entity_map[eval_entity_labels[i][j]])
        true_vectors.append(entity_vector_dict[int_to_entity_map[eval_entity_labels[i][j]]])
        predicted_vectors.append(eval_ed_outputs[i][j].cpu().numpy())
  # print(len(true_labels))
  entity_predictions = []
  for i in range(len(true_labels)):
    if(len(dataset_candidate_entities[i]) == 0):
      print("NO CANDIDATES CASE: ")
      entity_predictions.append("NOCANDIDATES")
      continue
    
    candidate_vectors = get_candidates_vectors(dataset_candidate_entities[i])
    sims = cosine_similarity(candidate_vectors,[predicted_vectors[i]])
    most_similar_candidate_index = np.argmax(sims)
    max_score = np.max(sims)
    most_similar_candidate = dataset_candidate_entities[i][most_similar_candidate_index]
    entity_predictions.append(most_similar_candidate)

    # # DEBUG
    # real_label_cos = cosine_similarity([true_vectors[0]],[predicted_vectors[i]])[0][0]
    # if(most_similar_candidate != true_labels[i]):
    #   print('DEBUG: REAL: {}, {}\tPREDICTED: {}, {}'.format(true_labels[i],real_label_cos,most_similar_candidate,max_score))

  return true_labels,entity_predictions

# ED Model

In [ ]:
# wiki2vec entity dimensions #TODO try other sizes 
entity_dim = 768

hidden_dropout_prob = 0.1
bert_hidden_size = 768

# LEFT AND RIGHT
CONTEXT_SIZE = 10

bert = BertModel.from_pretrained('bert-base-cased',add_pooling_layer=False).cuda()

# hidden_layer = torch.nn.Linear(hidden_size, entity_dim).cuda()

# ed_dropout = torch.nn.Dropout(hidden_dropout_prob).cuda()
# ed_mapping_layer = torch.nn.Linear(bert_hidden_size, entity_dim).cuda()


# +1 for the mention averaged vector
context_layer_input_dim = bert_hidden_size*(CONTEXT_SIZE+CONTEXT_SIZE+1)
context_layer = torch.nn.Linear(context_layer_input_dim, entity_dim).cuda()

# TODO add prior and hyperlink form? 
# contextvec + entityvec+ dotProduct 
ranking_layer_input_dim = entity_dim + entity_dim + 1 
# ranking_layer_input_dim = entity_dim + entity_dim + 1 + 1

ranking_layer = torch.nn.Linear(ranking_layer_input_dim, 1).cuda()


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
ranking_layer.load_state_dict(torch.load("drive/My Drive/ranking_layerCONTEXT20TOP40.hdf5"))
context_layer.load_state_dict(torch.load("drive/My Drive/context_layerCONTEXT20TOP40.hdf5"))
bert.load_state_dict(torch.load( "drive/My Drive/bertCONTEXT20TOP40.hdf5"))


<All keys matched successfully>

In [ ]:
# torch.save(ed_mapping_layer.state_dict(), "drive/My Drive/ed_layerCONTEXT40.hdf5")
torch.save(ranking_layer.state_dict(), "drive/My Drive/ranking_layerCONTEXT20TOP10PRIOR.hdf5")
torch.save(context_layer.state_dict(), "drive/My Drive/context_layerCONTEXT20TOP10PRIOR.hdf5")
torch.save(bert.state_dict(), "drive/My Drive/bertCONTEXT20TOP10PRIOR.hdf5")


In [ ]:
learning_rate = 2e-5
# params_to_optimize = list(ed_mapping_layer.parameters()) +list(bert.parameters()) + list(hidden_layer.parameters())
params_to_optimize = list(bert.parameters()) + list(context_layer.parameters()) + list(ranking_layer.parameters())
# params_to_optimize = list(ed_mapping_layer.parameters()) +list(bert.parameters())

optimizer_ed = AdamW(params_to_optimize, lr=learning_rate)

# tanh = torch.nn.Tanh()
relu = torch.nn.ReLU()

# cos_sim = torch.nn.CosineSimilarity()
# cross_ent = torch.nn.CrossEntropyLoss().cuda()
# hinge_embed_loss_fct = torch.nn.HingeEmbeddingLoss()
# cosine_loss_fct = torch.nn.CosineEmbeddingLoss(reduction="sum")

In [ ]:
'''
Note
that Lmd is calculated for all non-pad head WordPiece tokens but Led is calculated only for the first
WordPiece token of every labeled entity mention
with a linkable and valid entity ID label.
'''


In [ ]:
batch_size = 4
Epochs = 40
# NEGATIVE_EXAMPLES_COUNT = 3

training_steps = int(len(train_samples)/batch_size)+1

for epoch in range(Epochs):

  random.shuffle(train_samples)
  print("Epoch ", epoch)
  # global_entity_counter = 0
  losses = []
  with tqdm(total=training_steps) as progress_bar:
    for i in range(0, len(train_samples), batch_size):
        bert.train()
        context_layer.train()
        ranking_layer.train()
        # hidden_layer.train()

        batch_sentences = [sample.sentence for sample in train_samples[i:i+batch_size]]
        batch_ner_labels = [sample.iob_labels for sample in train_samples[i:i+batch_size]]
        batch_entities = [sample.entities for sample in train_samples[i:i+batch_size]]
        batch_ed = [sample.ed_labels for sample in train_samples[i:i+batch_size]]

        inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
        batch_ner_y = torch.LongTensor(inputs['labels']).cuda()
        batch_ed_label_y = torch.LongTensor(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map)).cuda()
        batch_ed_vector_y = get_entity_vectors(batch_ed_label_y,entity_vector_dict,int_to_entity_map).cuda()
        entity_locations,number_of_entities = get_entity_locations(batch_ner_y)

        # not all spans have negative samples
        # batch_ed_posneg_vector_x,batch_ed_posneg_y,num_of_examples = get_entity_vectors_v2(batch_ed)
        batch_ed_posneg_vector_x,batch_ed_posneg_prior_x,batch_ed_posneg_y,num_of_examples = get_entity_vectors_v3(batch_ed)
        



        outputs = bert(
              inputs['input_ids'].cuda(),
              attention_mask=inputs['attention_mask'].cuda(),
              token_type_ids=inputs['token_type_ids'].cuda()
          )

        
        counter = 0
        ED_outputs = torch.zeros((num_of_examples)).cuda()
        ED_labels = torch.zeros((num_of_examples)).cuda()
        for ex_i in range(len(batch_ed_vector_y)):
          for span_index,(sample_entity_location,span_size) in enumerate(entity_locations[ex_i]):

            context_start = sample_entity_location - CONTEXT_SIZE
            context_end = sample_entity_location + CONTEXT_SIZE

            # ensure a 2*CONTEXT_SIZE sequence
            if (context_start < 0):
              context_start = 0
              
              context_end = CONTEXT_SIZE+CONTEXT_SIZE
              
            # ensure a 2*CONTEXT_SIZE sequence
            if (context_end >= max_len):

              context_start = max_len-CONTEXT_SIZE-CONTEXT_SIZE
              context_end = max_len
              
            assert outputs[0][ex_i][context_start:context_end].shape[0] == 2*CONTEXT_SIZE

            context_vector = outputs[0][ex_i][context_start:context_end]

            #TODO
            if (span_size == 1):
              mention_vector = outputs[0][ex_i][sample_entity_location]

            # avg mention embeddings
            else:
              mention_vector = torch.mean(outputs[0][ex_i][sample_entity_location:(sample_entity_location+span_size)],dim=0)

            mention_vector = mention_vector.view(-1, entity_dim)
            concat_vec = torch.cat((mention_vector,context_vector)).view(-1)
            
            context_output = context_layer(concat_vec)


            ## loop for each pos neg example 
            ## should be NEG + 1 POS
            for span_example_index in range(len(batch_ed_posneg_vector_x[ex_i][span_index])):
              entity_vector = batch_ed_posneg_vector_x[ex_i][span_index][span_example_index].cuda()
              entity_label = torch.tensor(batch_ed_posneg_y[ex_i][span_index][span_example_index])
              entity_prior = batch_ed_posneg_prior_x[ex_i][span_index][span_example_index].cuda()

              dot_sim = torch.dot(context_output,entity_vector)

              combined_vec = torch.cat((context_output,entity_vector)).view(-1)
              combined_vec = torch.cat((combined_vec,dot_sim.view(1)))
              combined_vec = torch.cat((combined_vec,entity_prior.view(1)))

              ranking_out = ranking_layer(combined_vec)

              ED_outputs[counter] = ranking_out
              ED_labels[counter] = entity_label

              counter += 1

        
        loss1 = ED_labels * relu(0.2 - ED_outputs)
        loss2 = (1 - ED_labels) * relu(ED_outputs)
        loss = loss1 + loss2
        loss = torch.sum(loss)

        # torch.nn.utils.clip_grad_norm_(ed_mapping_layer.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(context_layer.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(ranking_layer.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(bert.parameters(), 1.0)

        # skip sequences with no entities (no loss)
        if (number_of_entities > 0):
          loss.backward()
          optimizer_ed.step()
          optimizer_ed.zero_grad()
          losses.append(loss.item())
        
        avg_loss = sum(losses)/len(losses)
        progress_bar.set_description("avg loss so far = {}".format(avg_loss))
        progress_bar.update(1)
        


    if(epoch % 3 == 0):
      eval_ed_outputs = []
      eval_entity_labels = []
      eval_sentences = []
      val_losses=[]
      bert.eval()
      # ed_mapping_layer.eval()
      context_layer.eval()
      ranking_layer.eval()
      # ed_dropout.eval()
      # hidden_layer.eval()

      val_steps = int(len(val_samples)/batch_size)+1
      with tqdm(total=val_steps) as progress_bar:
        for i in range(0, len(val_samples), batch_size):

            with torch.no_grad():
              batch_sentences = [sample.sentence for sample in val_samples[i:i+batch_size]]
              batch_ner_labels = [sample.iob_labels for sample in val_samples[i:i+batch_size]]
              batch_entities = [sample.entities for sample in val_samples[i:i+batch_size]]
              batch_ed = [sample.ed_labels for sample in val_samples[i:i+batch_size]]

              inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
              batch_ner_y = torch.LongTensor(inputs['labels']).cuda()
              batch_ed_label_y = torch.LongTensor(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map)).cuda()
              batch_ed_vector_y = get_entity_vectors(batch_ed_label_y,entity_vector_dict,int_to_entity_map).cuda()
              entity_locations,number_of_entities = get_entity_locations(batch_ner_y)
              # batch_ed_posneg_vector_x,batch_ed_posneg_y,num_of_examples = get_entity_vectors_v2(batch_ed)
              batch_ed_posneg_vector_x,batch_ed_posneg_prior_x,batch_ed_posneg_y,num_of_examples = get_entity_vectors_v3(batch_ed)



              outputs = bert(
                    inputs['input_ids'].cuda(),
                    attention_mask=inputs['attention_mask'].cuda(),
                    token_type_ids=inputs['token_type_ids'].cuda()
                )




              counter = 0
              # ED_outputs = torch.zeros((num_of_examples)).cuda()
              ED_outputs = []
              ED_labels = []
              # ED_labels = torch.zeros((num_of_examples)).cuda()
              for ex_i in range(len(batch_ed_vector_y)):
                for span_index,(sample_entity_location,span_size) in enumerate(entity_locations[ex_i]):

                  context_start = sample_entity_location - CONTEXT_SIZE
                  context_end = sample_entity_location + CONTEXT_SIZE

                  # ensure a 2*CONTEXT_SIZE sequence
                  if (context_start < 0):
                    context_start = 0
                    
                    context_end = CONTEXT_SIZE+CONTEXT_SIZE
                    
                  # ensure a 2*CONTEXT_SIZE sequence
                  if (context_end >= max_len):

                    context_start = max_len-CONTEXT_SIZE-CONTEXT_SIZE
                    context_end = max_len
                    
                  assert outputs[0][ex_i][context_start:context_end].shape[0] == 2*CONTEXT_SIZE

                  context_vector = outputs[0][ex_i][context_start:context_end]

                  #TODO
                  if (span_size == 1):
                    mention_vector = outputs[0][ex_i][sample_entity_location]

                  # avg mention embeddings
                  else:
                    mention_vector = torch.mean(outputs[0][ex_i][sample_entity_location:(sample_entity_location+span_size)],dim=0)

                  mention_vector = mention_vector.view(-1, entity_dim)
                  concat_vec = torch.cat((mention_vector,context_vector)).view(-1)
                  
                  context_output = context_layer(concat_vec)


                  ## loop for each pos neg example 
                  ## should be NEG + 1 POS
                  span_outputs = []
                  span_labels = []
                  for span_example_index in range(len(batch_ed_posneg_vector_x[ex_i][span_index])):
                    entity_vector = batch_ed_posneg_vector_x[ex_i][span_index][span_example_index].cuda()
                    entity_label = torch.tensor(batch_ed_posneg_y[ex_i][span_index][span_example_index])


                    dot_sim = torch.dot(context_output,entity_vector)

                    combined_vec = torch.cat((context_output,entity_vector)).view(-1)
                    combined_vec = torch.cat((combined_vec,dot_sim.view(1)))

                    entity_prior = batch_ed_posneg_prior_x[ex_i][span_index][span_example_index].cuda()
                    combined_vec = torch.cat((combined_vec,entity_prior.view(1)))


                    ranking_out = ranking_layer(combined_vec)

                    span_outputs.append(ranking_out.item())
                    span_labels.append(entity_label.item())
                    # ED_outputs[counter] = ranking_out
                    # ED_labels[counter] = entity_label

                    counter += 1
                  ED_outputs.append(span_outputs)
                  ED_labels.append(span_labels)
              # loss1 = ED_labels * relu(0.2 - ED_outputs)
              # loss2 = (1 - ED_labels) * relu(ED_outputs)
              # val_loss = loss1 + loss2
              # val_losses.append(torch.sum(val_loss).item())


              progress_bar.update(1)
              eval_ed_outputs.extend(ED_outputs)
              eval_entity_labels.extend(ED_labels)
      errors = 0
      for span_outputs,span_labels in zip(eval_ed_outputs,eval_entity_labels):
        if np.argmax(np.array(span_outputs)) != np.argmax(np.array(span_labels)):
          errors +=1
      print(1-(errors/len(eval_ed_outputs)))
                    
      # print("val loss",sum(val_losses))

  0%|          | 0/274 [00:00<?, ?it/s]

Epoch  0


  0%|          | 0/274 [00:00<?, ?it/s]

0.1361256544502618
Epoch  1


  0%|          | 0/274 [00:00<?, ?it/s]

...

Epoch  38


  0%|          | 0/274 [00:00<?, ?it/s]

Epoch  39


avg loss so far = 1.6074542627010466: 100%|██████████| 274/274 [07:50<00:00,  1.72s/it]

0.9145549738219896


In [ ]:
eval_ed_outputs = []
eval_entity_labels = []
eval_sentences = []
val_losses=[]
bert.eval()
# ed_mapping_layer.eval()
context_layer.eval()
ranking_layer.eval()
# ed_dropout.eval()
# hidden_layer.eval()
batch_size = 4


val_steps = int(len(val_samples)/batch_size)+1
with tqdm(total=val_steps) as progress_bar:
  for i in range(0, len(val_samples), batch_size):

      with torch.no_grad():
        batch_sentences = [sample.sentence for sample in val_samples[i:i+batch_size]]
        batch_ner_labels = [sample.iob_labels for sample in val_samples[i:i+batch_size]]
        batch_entities = [sample.entities for sample in val_samples[i:i+batch_size]]
        batch_ed = [sample.ed_labels for sample in val_samples[i:i+batch_size]]

        inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
        batch_ner_y = torch.LongTensor(inputs['labels']).cuda()
        batch_ed_label_y = torch.LongTensor(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map)).cuda()
        batch_ed_vector_y = get_entity_vectors(batch_ed_label_y,entity_vector_dict,int_to_entity_map).cuda()
        entity_locations,number_of_entities = get_entity_locations(batch_ner_y)
        # batch_ed_posneg_vector_x,batch_ed_posneg_y,num_of_examples = get_entity_vectors_v2(batch_ed)
        batch_ed_posneg_vector_x,batch_ed_posneg_prior_x,batch_ed_posneg_y,num_of_examples = get_entity_vectors_v3(batch_ed)



        outputs = bert(
              inputs['input_ids'].cuda(),
              attention_mask=inputs['attention_mask'].cuda(),
              token_type_ids=inputs['token_type_ids'].cuda()
          )




        counter = 0
        # ED_outputs = torch.zeros((num_of_examples)).cuda()
        ED_outputs = []
        ED_labels = []
        # ED_labels = torch.zeros((num_of_examples)).cuda()
        for ex_i in range(len(batch_ed_vector_y)):
          for span_index,(sample_entity_location,span_size) in enumerate(entity_locations[ex_i]):

            context_start = sample_entity_location - CONTEXT_SIZE
            context_end = sample_entity_location + CONTEXT_SIZE

            # ensure a 2*CONTEXT_SIZE sequence
            if (context_start < 0):
              context_start = 0
              
              context_end = CONTEXT_SIZE+CONTEXT_SIZE
              
            # ensure a 2*CONTEXT_SIZE sequence
            if (context_end >= max_len):

              context_start = max_len-CONTEXT_SIZE-CONTEXT_SIZE
              context_end = max_len
              
            assert outputs[0][ex_i][context_start:context_end].shape[0] == 2*CONTEXT_SIZE

            context_vector = outputs[0][ex_i][context_start:context_end]

            #TODO
            if (span_size == 1):
              mention_vector = outputs[0][ex_i][sample_entity_location]

            # avg mention embeddings
            else:
              mention_vector = torch.mean(outputs[0][ex_i][sample_entity_location:(sample_entity_location+span_size)],dim=0)

            mention_vector = mention_vector.view(-1, entity_dim)
            concat_vec = torch.cat((mention_vector,context_vector)).view(-1)
            
            context_output = context_layer(concat_vec)

            ## loop for each pos neg example 
            ## should be NEG + 1 POS
            span_outputs = []
            span_labels = []
            for span_example_index in range(len(batch_ed_posneg_vector_x[ex_i][span_index])):
              entity_vector = batch_ed_posneg_vector_x[ex_i][span_index][span_example_index].cuda()
              entity_label = torch.tensor(batch_ed_posneg_y[ex_i][span_index][span_example_index])

              dot_sim = torch.dot(context_output,entity_vector)

              combined_vec = torch.cat((context_output,entity_vector)).view(-1)
              combined_vec = torch.cat((combined_vec,dot_sim.view(1)))

              entity_prior = batch_ed_posneg_prior_x[ex_i][span_index][span_example_index].cuda()
              combined_vec = torch.cat((combined_vec,entity_prior.view(1)))
              ranking_out = ranking_layer(combined_vec)

              span_outputs.append(ranking_out.item())
              span_labels.append(entity_label.item())
              # ED_outputs[counter] = ranking_out
              # ED_labels[counter] = entity_label

              counter += 1
            ED_outputs.append(span_outputs)
            ED_labels.append(span_labels)
        # loss1 = ED_labels * relu(0.2 - ED_outputs)
        # loss2 = (1 - ED_labels) * relu(ED_outputs)
        # val_loss = loss1 + loss2
        # val_losses.append(torch.sum(val_loss).item())


        progress_bar.update(1)
        eval_ed_outputs.extend(ED_outputs)
        eval_entity_labels.extend(ED_labels)
errors = 0
for span_outputs,span_labels in zip(eval_ed_outputs,eval_entity_labels):
  if np.argmax(np.array(span_outputs)) != np.argmax(np.array(span_labels)):
    errors +=1
print(1-(errors/len(eval_ed_outputs)))
              
# print("val loss",sum(val_losses))

 99%|█████████▊| 68/69 [01:06<00:00,  1.03it/s]

0.9145549738219896


In [ ]:
eval_ed_outputs = []
eval_entity_labels = []
eval_sentences = []
val_losses=[]
bert.eval()
# ed_mapping_layer.eval()
context_layer.eval()
ranking_layer.eval()
# ed_dropout.eval()
# hidden_layer.eval()
batch_size = 4


val_steps = int(len(test_samples)/batch_size)+1
with tqdm(total=val_steps) as progress_bar:
  for i in range(0, len(test_samples), batch_size):

      with torch.no_grad():
        batch_sentences = [sample.sentence for sample in test_samples[i:i+batch_size]]
        batch_ner_labels = [sample.iob_labels for sample in test_samples[i:i+batch_size]]
        batch_entities = [sample.entities for sample in test_samples[i:i+batch_size]]
        batch_ed = [sample.ed_labels for sample in test_samples[i:i+batch_size]]

        inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
        batch_ner_y = torch.LongTensor(inputs['labels']).cuda()
        batch_ed_label_y = torch.LongTensor(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map)).cuda()
        batch_ed_vector_y = get_entity_vectors(batch_ed_label_y,entity_vector_dict,int_to_entity_map).cuda()
        entity_locations,number_of_entities = get_entity_locations(batch_ner_y)
        batch_ed_posneg_vector_x,batch_ed_posneg_y,num_of_examples = get_entity_vectors_v2(batch_ed)
        # batch_ed_posneg_vector_x,batch_ed_posneg_prior_x,batch_ed_posneg_y,num_of_examples = get_entity_vectors_v3(batch_ed)



        outputs = bert(
              inputs['input_ids'].cuda(),
              attention_mask=inputs['attention_mask'].cuda(),
              token_type_ids=inputs['token_type_ids'].cuda()
          )




        counter = 0
        # ED_outputs = torch.zeros((num_of_examples)).cuda()
        ED_outputs = []
        ED_labels = []
        # ED_labels = torch.zeros((num_of_examples)).cuda()
        for ex_i in range(len(batch_ed_vector_y)):
          for span_index,(sample_entity_location,span_size) in enumerate(entity_locations[ex_i]):

            context_start = sample_entity_location - CONTEXT_SIZE
            context_end = sample_entity_location + CONTEXT_SIZE

            # ensure a 2*CONTEXT_SIZE sequence
            if (context_start < 0):
              context_start = 0
              
              context_end = CONTEXT_SIZE+CONTEXT_SIZE
              
            # ensure a 2*CONTEXT_SIZE sequence
            if (context_end >= max_len):

              context_start = max_len-CONTEXT_SIZE-CONTEXT_SIZE
              context_end = max_len
              
            assert outputs[0][ex_i][context_start:context_end].shape[0] == 2*CONTEXT_SIZE

            context_vector = outputs[0][ex_i][context_start:context_end]

            #TODO
            if (span_size == 1):
              mention_vector = outputs[0][ex_i][sample_entity_location]

            # avg mention embeddings
            else:
              mention_vector = torch.mean(outputs[0][ex_i][sample_entity_location:(sample_entity_location+span_size)],dim=0)

            mention_vector = mention_vector.view(-1, entity_dim)
            concat_vec = torch.cat((mention_vector,context_vector)).view(-1)
            
            context_output = context_layer(concat_vec)

            ## loop for each pos neg example 
            ## should be NEG + 1 POS
            span_outputs = []
            span_labels = []
            for span_example_index in range(len(batch_ed_posneg_vector_x[ex_i][span_index])):
              entity_vector = batch_ed_posneg_vector_x[ex_i][span_index][span_example_index].cuda()
              entity_label = torch.tensor(batch_ed_posneg_y[ex_i][span_index][span_example_index])

              dot_sim = torch.dot(context_output,entity_vector)

              combined_vec = torch.cat((context_output,entity_vector)).view(-1)
              combined_vec = torch.cat((combined_vec,dot_sim.view(1)))

              # entity_prior = batch_ed_posneg_prior_x[ex_i][span_index][span_example_index].cuda()
              # combined_vec = torch.cat((combined_vec,entity_prior.view(1)))
              ranking_out = ranking_layer(combined_vec)

              span_outputs.append(ranking_out.item())
              span_labels.append(entity_label.item())
              # ED_outputs[counter] = ranking_out
              # ED_labels[counter] = entity_label

              counter += 1
            ED_outputs.append(span_outputs)
            ED_labels.append(span_labels)
        # loss1 = ED_labels * relu(0.2 - ED_outputs)
        # loss2 = (1 - ED_labels) * relu(ED_outputs)
        # val_loss = loss1 + loss2
        # val_losses.append(torch.sum(val_loss).item())


        progress_bar.update(1)
        eval_ed_outputs.extend(ED_outputs)
        eval_entity_labels.extend(ED_labels)
errors = 0
for span_outputs,span_labels in zip(eval_ed_outputs,eval_entity_labels):
  if np.argmax(np.array(span_outputs)) != np.argmax(np.array(span_labels)):
    errors +=1
print(1-(errors/len(eval_ed_outputs)))
              
# print("val loss",sum(val_losses))

100%|██████████| 67/67 [00:59<00:00,  1.12it/s]

0.8056734420370784


In [ ]:
eval_ed_outputs = []
eval_entity_labels = []
eval_sentences = []

bert.eval()
ed_mapping_layer.eval()
# ed_dropout.eval()
# hidden_layer.eval()

val_steps = int(len(val_samples)/batch_size)+1
with tqdm(total=val_steps) as progress_bar:
  for i in range(0, len(val_samples), batch_size):

      with torch.no_grad():
        batch_sentences = [sample.sentence for sample in val_samples[i:i+batch_size]]
        batch_sentences = [sample.sentence for sample in val_samples[i:i+batch_size]]
        batch_ner_labels = [sample.iob_labels for sample in val_samples[i:i+batch_size]]
        batch_entities = [sample.entities for sample in val_samples[i:i+batch_size]]

        inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
        batch_ner_y = torch.LongTensor(inputs['labels']).cuda()
        batch_ed_label_y = torch.LongTensor(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map)).cuda()
        batch_ed_vector_y = get_entity_vectors(batch_ed_label_y,entity_vector_dict,int_to_entity_map).cuda()
        entity_locations,number_of_entities = get_entity_locations(batch_ner_y)


        outputs = bert(
              inputs['input_ids'].cuda(),
              attention_mask=inputs['attention_mask'].cuda(),
              token_type_ids=inputs['token_type_ids'].cuda()
          )




        counter = 0
        predicted_entities = torch.zeros((number_of_entities,entity_dim)).cuda()
        for ex_i in range(len(batch_ed_vector_y)):
          for sample_entity_location,span_size in entity_locations[ex_i]:
            if (span_size == 1):
              predicted_entities[counter] = outputs[0][ex_i][sample_entity_location]

            # avg mention embeddings
            else:
              predicted_entities[counter] = torch.mean(outputs[0][ex_i][sample_entity_location:(sample_entity_location+span_size)],dim=0)

            counter += 1

        ed_logits = ed_mapping_layer(predicted_entities)

        ed_logits = tanh(ed_logits)

        counter = 0
        pred_raw_outputs = torch.zeros((batch_size,max_len,entity_dim)).cuda()
        for ex_i in range(len(batch_ed_vector_y)):
          for sample_entity_location,span_size in entity_locations[ex_i]:
            
            pred_raw_outputs[ex_i][sample_entity_location] = ed_logits[counter]

            counter+=1


        eval_sentences.extend(batch_sentences)
        eval_ed_outputs.extend(pred_raw_outputs)
        eval_entity_labels.extend(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map))
        progress_bar.update(1)
distances = ED_avg_cos_dis(eval_entity_labels,eval_ed_outputs)
print('Distances: ',np.mean(distances))
true_labels,entity_predictions = ED_eval(eval_entity_labels,eval_ed_outputs,val_candidate_entities)
print('acc: ',accuracy_score(true_labels,entity_predictions))

In [ ]:
true_labels,entity_predictions = ED_eval(eval_entity_labels,eval_ed_outputs,val_candidate_entities)
print('acc: ',accuracy_score(true_labels,entity_predictions))

DEBUG: REAL: Chester-le-Street, 0.8700860738754272	PREDICTED: Chester, 0.6964319944381714
DEBUG: REAL: 1998_FIFA_World_Cup, 0.9883451461791992	PREDICTED: 2010_FIFA_World_Cup, 0.7333267331123352
DEBUG: REAL: Alan_Shearer, 0.906802773475647	PREDICTED: Alan_Jackson_(EastEnders), 0.8482780456542969
DEBUG: REAL: Alan_Shearer, 0.905540943145752	PREDICTED: Alan_Osório_da_Costa_Silva, 0.8512593507766724
DEBUG: REAL: Los_Angeles_Angels_of_Anaheim, 0.9990792870521545	PREDICTED: 2008_California_Golden_Bears_football_team, 0.6847493648529053
DEBUG: REAL: New_York_Yankees, 0.9987706542015076	PREDICTED: Harry_Stewart_New, 0.583346426486969
DEBUG: REAL: Chicago_White_Sox, 0.9988444447517395	PREDICTED: Chicago_Cubs, 0.7329671382904053
DEBUG: REAL: Los_Angeles_Dodgers, 0.991845428943634	PREDICTED: Los_Angeles_Lakers, 0.7376371622085571
DEBUG: REAL: Mauro_Silva, 0.9641743898391724	PREDICTED: Silva_Batuta, 0.9539585113525391
DEBUG: REAL: Dennis_Mitchell, 0.9684586524963379	PREDICTED: Jared_Mitchell_(base

In [ ]:
144 / len(true_labels)

0.030157068062827225

## ed with mention averaging


In [ ]:
batch_size = 4
Epochs = 10
# NEGATIVE_EXAMPLES_COUNT = 3

training_steps = int(len(train_samples)/batch_size)+1

for epoch in range(Epochs):

  random.shuffle(train_samples)
  print("Epoch ", epoch)
  # global_entity_counter = 0
  losses = []
  with tqdm(total=training_steps) as progress_bar:
    for i in range(0, len(train_samples), batch_size):
        bert.train()
        ed_mapping_layer.train()
        # ed_dropout.train()
        # hidden_layer.train()

        batch_sentences = [sample.sentence for sample in train_samples[i:i+batch_size]]
        batch_ner_labels = [sample.iob_labels for sample in train_samples[i:i+batch_size]]
        batch_entities = [sample.entities for sample in train_samples[i:i+batch_size]]
        batch_ed = [sample.ed_labels for sample in train_samples[i:i+batch_size]]

        inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
        batch_ner_y = torch.LongTensor(inputs['labels']).cuda()
        batch_ed_label_y = torch.LongTensor(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map)).cuda()
        batch_ed_vector_y = get_entity_vectors(batch_ed_label_y,entity_vector_dict,int_to_entity_map).cuda()
        entity_locations,number_of_entities = get_entity_locations(batch_ner_y)

        # not all spans have negative samples
        # batch_ed_posneg_vector_x,batch_ed_posneg_vector_y,num_of_examples = get_entity_vectors_v2(batch_ed)
        



        outputs = bert(
              inputs['input_ids'].cuda(),
              attention_mask=inputs['attention_mask'].cuda(),
              token_type_ids=inputs['token_type_ids'].cuda()
          )

        
        

        # hidden_layer_output = hidden_layer(tanh(outputs[0]))
        
        # dropped_out = ed_dropout(hidden_layer_output)


        mask = batch_ed_label_y.unsqueeze(-1).expand_as(outputs[0]).ge(0)

        # masked_out = torch.masked_select(ed_logits, mask)
        # masked_out = masked_out.view(-1, entity_dim) 

        masked_labels = torch.masked_select(batch_ed_vector_y, mask)
        masked_labels = masked_labels.view(-1, entity_dim) 

        counter = 0
        predicted_entities = torch.zeros((number_of_entities,entity_dim)).cuda()
        for ex_i in range(len(batch_ed_vector_y)):
          for sample_entity_location,span_size in entity_locations[ex_i]:
            if (span_size == 1):
              predicted_entities[counter] = outputs[0][ex_i][sample_entity_location]

            # avg mention embeddings
            else:
              predicted_entities[counter] = torch.mean(outputs[0][ex_i][sample_entity_location:(sample_entity_location+span_size)],dim=0)

            counter += 1

        ed_logits = ed_mapping_layer(predicted_entities)

        ed_logits = tanh(ed_logits)
        
        ed_logits = ed_logits.view(-1, entity_dim) 

        target = masked_labels.new(masked_labels.size(0)).fill_(1) 

        loss = cosine_loss_fct(ed_logits,masked_labels,target)
            
        # single_losses = torch.zeros(number_of_entities*(NEGATIVE_EXAMPLES_COUNT+1)).cuda()
        # temp = []
        # counter = 0

        # for ex_i in range(len(batch_ed_vector_y)):
        #   for sample_entity_location in entity_locations[ex_i]:
        #     single_loss = 1 - torch.abs(cos_sim(ed_logits[ex_i][sample_entity_location].view(-1,entity_dim),batch_ed_vector_y[ex_i][sample_entity_location].view(-1,entity_dim)))
        #     single_losses[counter] = single_loss
        #     counter += 1
        #     # negative examples loss
        #     if(len(train_candidate_entities[global_entity_counter]) >= NEGATIVE_EXAMPLES_COUNT):
        #       candidate_vectors = get_candidates_vectors(train_candidate_entities[global_entity_counter])
        #       # print('OG entity',int_to_entity_map[batch_ed_label_y[ex_i][sample_entity_location].item()])
        #       # print('can entities' , train_candidate_entities[global_entity_counter])
              
        #       for negative_example_index in range(NEGATIVE_EXAMPLES_COUNT):
        #         if (int_to_entity_map[batch_ed_label_y[ex_i][sample_entity_location].item()] != train_candidate_entities[global_entity_counter][negative_example_index]):

        #           neg_vector = torch.tensor(candidate_vectors[negative_example_index]).cuda().view(-1,entity_dim)
        #           # print(train_candidate_entities[global_entity_counter][negative_example_index])
        #           # print(neg_vector)
        #           neg_loss = torch.abs(cos_sim(ed_logits[ex_i][sample_entity_location].view(-1,entity_dim),neg_vector))
        #           temp.append(neg_loss)
        #           single_losses[counter] = neg_loss
        #           counter += 1
        #           # print(neg_loss)
        #           # print(single_losses)


            
        #     global_entity_counter += 1

        # loss = torch.mean(single_losses[:counter])



        # TODO should I clip bert aswell?
        # torch.nn.utils.clip_grad_norm_(hidden_layer.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(ed_mapping_layer.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(bert.parameters(), 1.0)

        # skip sequences with no entities (no loss)
        if (number_of_entities > 0):
          loss.backward()
          optimizer_ed.step()
          optimizer_ed.zero_grad()
          losses.append(loss.item())
        
        avg_loss = sum(losses)/len(losses)
        progress_bar.update(1)
        progress_bar.set_description("avg loss so far = {}".format(avg_loss))


    if(epoch % 3 == 0):
      eval_ed_outputs = []
      eval_entity_labels = []
      eval_sentences = []

      bert.eval()
      ed_mapping_layer.eval()
      # ed_dropout.eval()
      # hidden_layer.eval()

      val_steps = int(len(val_samples)/batch_size)+1
      with tqdm(total=val_steps) as progress_bar:
        for i in range(0, len(val_samples), batch_size):

            with torch.no_grad():
              batch_sentences = [sample.sentence for sample in val_samples[i:i+batch_size]]
              batch_sentences = [sample.sentence for sample in val_samples[i:i+batch_size]]
              batch_ner_labels = [sample.iob_labels for sample in val_samples[i:i+batch_size]]
              batch_entities = [sample.entities for sample in val_samples[i:i+batch_size]]

              inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
              batch_ner_y = torch.LongTensor(inputs['labels']).cuda()
              batch_ed_label_y = torch.LongTensor(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map)).cuda()
              batch_ed_vector_y = get_entity_vectors(batch_ed_label_y,entity_vector_dict,int_to_entity_map).cuda()
              entity_locations,number_of_entities = get_entity_locations(batch_ner_y)


              outputs = bert(
                    inputs['input_ids'].cuda(),
                    attention_mask=inputs['attention_mask'].cuda(),
                    token_type_ids=inputs['token_type_ids'].cuda()
                )




              counter = 0
              predicted_entities = torch.zeros((number_of_entities,entity_dim)).cuda()
              for ex_i in range(len(batch_ed_vector_y)):
                for sample_entity_location,span_size in entity_locations[ex_i]:
                  if (span_size == 1):
                    predicted_entities[counter] = outputs[0][ex_i][sample_entity_location]

                  # avg mention embeddings
                  else:
                    predicted_entities[counter] = torch.mean(outputs[0][ex_i][sample_entity_location:(sample_entity_location+span_size)],dim=0)

                  counter += 1

              ed_logits = ed_mapping_layer(predicted_entities)

              ed_logits = tanh(ed_logits)

              counter = 0
              pred_raw_outputs = torch.zeros((batch_size,max_len,entity_dim)).cuda()
              for ex_i in range(len(batch_ed_vector_y)):
                for sample_entity_location,span_size in entity_locations[ex_i]:
                  
                  pred_raw_outputs[ex_i][sample_entity_location] = ed_logits[counter]

                  counter+=1


              eval_sentences.extend(batch_sentences)
              eval_ed_outputs.extend(pred_raw_outputs)
              eval_entity_labels.extend(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map))
              progress_bar.update(1)
      distances = ED_avg_cos_dis(eval_entity_labels,eval_ed_outputs)
      print('Distances: ',np.mean(distances))
      true_labels,entity_predictions = ED_eval(eval_entity_labels,eval_ed_outputs,val_candidate_entities)
      print('acc: ',accuracy_score(true_labels,entity_predictions))

## ED OLD

In [ ]:
batch_size = 4
Epochs = 30 
# NEGATIVE_EXAMPLES_COUNT = 3

training_steps = int(len(train_samples)/batch_size)+1

for epoch in range(Epochs):

  random.shuffle(train_samples)
  print("Epoch ", epoch)
  # global_entity_counter = 0
  losses = []
  with tqdm(total=training_steps) as progress_bar:
    for i in range(0, len(train_samples), batch_size):
        bert.train()
        ed_mapping_layer.train()
        ed_dropout.train()
        hidden_layer.train()

        batch_sentences = [sample.sentence for sample in train_samples[i:i+batch_size]]
        batch_ner_labels = [sample.iob_labels for sample in train_samples[i:i+batch_size]]
        batch_entities = [sample.entities for sample in train_samples[i:i+batch_size]]

        inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
        batch_ner_y = torch.LongTensor(inputs['labels']).cuda()
        batch_ed_label_y = torch.LongTensor(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map)).cuda()
        batch_ed_vector_y = get_entity_vectors(batch_ed_label_y,entity_vector_dict,int_to_entity_map).cuda()
        entity_locations,number_of_entities = get_entity_locations(batch_ed_label_y)
        

        outputs = bert(
              inputs['input_ids'].cuda(),
              attention_mask=inputs['attention_mask'].cuda(),
              token_type_ids=inputs['token_type_ids'].cuda()
          )

        
        

        hidden_layer_output = hidden_layer(tanh(outputs[0]))
        
        dropped_out = ed_dropout(hidden_layer_output)
        ed_logits = ed_mapping_layer(dropped_out)

        ed_logits = tanh(ed_logits)

        mask = batch_ed_label_y.unsqueeze(-1).expand_as(ed_logits).ge(0)

        masked_out = torch.masked_select(ed_logits, mask)
        masked_out = masked_out.view(-1, entity_dim) 

        masked_labels = torch.masked_select(batch_ed_vector_y, mask)
        masked_labels = masked_labels.view(-1, entity_dim) 

        target = masked_out.new(masked_out.size(0)).fill_(1) 

        loss = cosine_loss_fct(masked_out,masked_labels,target)

        # single_losses = torch.zeros(number_of_entities*(NEGATIVE_EXAMPLES_COUNT+1)).cuda()
        # temp = []
        # counter = 0

        # for ex_i in range(len(batch_ed_vector_y)):
        #   for sample_entity_location in entity_locations[ex_i]:
        #     single_loss = 1 - torch.abs(cos_sim(ed_logits[ex_i][sample_entity_location].view(-1,entity_dim),batch_ed_vector_y[ex_i][sample_entity_location].view(-1,entity_dim)))
        #     single_losses[counter] = single_loss
        #     counter += 1
        #     # negative examples loss
        #     if(len(train_candidate_entities[global_entity_counter]) >= NEGATIVE_EXAMPLES_COUNT):
        #       candidate_vectors = get_candidates_vectors(train_candidate_entities[global_entity_counter])
        #       # print('OG entity',int_to_entity_map[batch_ed_label_y[ex_i][sample_entity_location].item()])
        #       # print('can entities' , train_candidate_entities[global_entity_counter])
              
        #       for negative_example_index in range(NEGATIVE_EXAMPLES_COUNT):
        #         if (int_to_entity_map[batch_ed_label_y[ex_i][sample_entity_location].item()] != train_candidate_entities[global_entity_counter][negative_example_index]):

        #           neg_vector = torch.tensor(candidate_vectors[negative_example_index]).cuda().view(-1,entity_dim)
        #           # print(train_candidate_entities[global_entity_counter][negative_example_index])
        #           # print(neg_vector)
        #           neg_loss = torch.abs(cos_sim(ed_logits[ex_i][sample_entity_location].view(-1,entity_dim),neg_vector))
        #           temp.append(neg_loss)
        #           single_losses[counter] = neg_loss
        #           counter += 1
        #           # print(neg_loss)
        #           # print(single_losses)


            
        #     global_entity_counter += 1

        # loss = torch.mean(single_losses[:counter])



        # TODO should I clip bert aswell?
        torch.nn.utils.clip_grad_norm_(hidden_layer.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(ed_mapping_layer.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(bert.parameters(), 1.0)

        # skip sequences with no entities (no loss)
        if (number_of_entities > 0):
          loss.backward()
          optimizer_ed.step()
          optimizer_ed.zero_grad()
          losses.append(loss.item())
        
        avg_loss = sum(losses)/len(losses)
        progress_bar.update(1)
        progress_bar.set_description("avg loss so far = {}".format(avg_loss))


    if(epoch % 3 == 0):
      eval_ed_outputs = []
      eval_entity_labels = []
      eval_sentences = []
      
      bert.eval()
      ed_mapping_layer.eval()
      ed_dropout.eval()
      hidden_layer.eval()

      val_steps = int(len(val_samples)/batch_size)+1
      with tqdm(total=val_steps) as progress_bar:
        for i in range(0, len(val_samples), batch_size):

            with torch.no_grad():
              batch_sentences = [sample.sentence for sample in val_samples[i:i+batch_size]]
              batch_sentences = [sample.sentence for sample in val_samples[i:i+batch_size]]
              batch_ner_labels = [sample.iob_labels for sample in val_samples[i:i+batch_size]]
              batch_entities = [sample.entities for sample in val_samples[i:i+batch_size]]

              inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
              batch_ner_y = torch.LongTensor(inputs['labels']).cuda()
              batch_ed_label_y = torch.LongTensor(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map)).cuda()
              batch_ed_vector_y = get_entity_vectors(batch_ed_label_y,entity_vector_dict,int_to_entity_map).cuda()

              outputs = bert(
                    inputs['input_ids'].cuda(),
                    attention_mask=inputs['attention_mask'].cuda(),
                    token_type_ids=inputs['token_type_ids'].cuda()
                )



              hidden_layer_output = hidden_layer(tanh(outputs[0]))
              
              dropped_out = ed_dropout(hidden_layer_output)
              ed_logits = ed_mapping_layer(dropped_out)
              ed_logits = tanh(ed_logits)
              
              eval_sentences.extend(batch_sentences)
              eval_ed_outputs.extend(ed_logits)
              eval_entity_labels.extend(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map))
              progress_bar.update(1)
      distances = ED_avg_cos_dis(eval_entity_labels,eval_ed_outputs)
      print('Distances: ',np.mean(distances))
      true_labels,entity_predictions = ED_eval(eval_entity_labels,eval_ed_outputs,val_candidate_entities)
      print('acc: ',accuracy_score(true_labels,entity_predictions))

  0%|          | 0/274 [00:00<?, ?it/s]

Epoch  0


avg loss so far = 24.870773741404214:  82%|████████▏ | 225/274 [01:39<00:21,  2.27it/s]


KeyboardInterrupt: ignored

In [ ]:
true_labels,entity_predictions = ED_eval(eval_entity_labels,eval_ed_outputs,val_candidate_entities)
print('acc: ',accuracy_score(true_labels,entity_predictions))

acc:  0.8102617801047121


In [ ]:
dataset_candidate_entities =  val_candidate_entities

# a list with predicted vectors
predicted_vectors = []
true_labels = []
true_vectors = []
#TODO all sets
for i in range(len(eval_entity_labels)):
  for j in range(max_len):
    if(eval_entity_labels[i][j] != -100):
      true_labels.append(int_to_entity_map[eval_entity_labels[i][j]])
      true_vectors.append(entity_vector_dict[int_to_entity_map[eval_entity_labels[i][j]]])
      predicted_vectors.append(eval_ed_outputs[i][j].cpu().numpy())
# print(len(true_labels))
entity_predictions = []
errors = []
for i in range(len(true_labels)):
  if(len(dataset_candidate_entities[i]) == 0):
    print("NO CANDIDATES CASE: ")
    entity_predictions.append("NOCANDIDATES")
    continue
  
  candidate_vectors = get_candidates_vectors(dataset_candidate_entities[i])
  sims = cosine_similarity(candidate_vectors,[predicted_vectors[i]])
  most_similar_candidate_index = np.argmax(sims)
  max_score = np.max(sims)
  most_similar_candidate = dataset_candidate_entities[i][most_similar_candidate_index]
  entity_predictions.append(most_similar_candidate)
  if(true_labels[i] !=most_similar_candidate ):
    errors.append((i,most_similar_candidate,true_labels[i] ,dataset_candidate_entities[i]))

In [ ]:
not_in_cands = 0
for error in errors:
  if(not error[2] in error[3] ):
    not_in_cands += 1
  
not_in_cands

194

In [ ]:
len(errors)

1141

In [ ]:
batch_size = 4
Epochs = 15
#todo do i need to set bert?
# bert.train()

# training_sentences = [value[0] for value in train_data.values()]
# training_ner_labels = [value[2] for value in train_data.values()]

training_steps = int(len(train_samples)/batch_size)+1
losses = []
for epoch in range(Epochs):
  with tqdm(total=training_steps) as progress_bar:
    for i in range(0, len(train_samples), batch_size):
        bert.train()
        ed_mapping_layer.train()
        ed_dropout.train()

        batch_sentences = [sample.sentence for sample in train_samples[i:i+batch_size]]
        batch_sentences = [sample.sentence for sample in train_samples[i:i+batch_size]]
        batch_ner_labels = [sample.iob_labels for sample in train_samples[i:i+batch_size]]
        batch_entities = [sample.entities for sample in train_samples[i:i+batch_size]]

        inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
        batch_ner_y = torch.LongTensor(inputs['labels']).cuda()
        batch_ed_label_y = torch.LongTensor(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map)).cuda()
        batch_ed_vector_y = get_entity_vectors(batch_ed_label_y,entity_vector_dict,int_to_entity_map).cuda()
        entity_locations,number_of_entities = get_entity_locations(batch_ed_label_y)

        outputs = bert(
              inputs['input_ids'].cuda(),
              attention_mask=inputs['attention_mask'].cuda(),
              token_type_ids=inputs['token_type_ids'].cuda()
          )

        
        sequence_output = ed_dropout(outputs[0])

        ed_logits = ed_mapping_layer(sequence_output)

        ed_logits = tanh(ed_logits)

        single_losses = torch.zeros(number_of_entities).cuda()
        counter = 0
        for ex_i in range(len(batch_ed_vector_y)):
          for sample_entity_location in entity_locations[ex_i]:
            single_loss = 1 - torch.abs(cos_sim(ed_logits[ex_i][sample_entity_location].view(-1,entity_dim),batch_ed_vector_y[ex_i][sample_entity_location].view(-1,entity_dim)))
            single_losses[counter] = single_loss
            counter += 1
        loss = torch.mean(single_losses)

        # loss =  torch.mean(single_losses)

        # TODO should I clip bert aswell?
        torch.nn.utils.clip_grad_norm_(ed_mapping_layer.parameters(), 1.0)
        loss.backward()
        optimizer_ed.step()
        optimizer_ed.zero_grad()

        losses.append(loss.item())
        avg_loss = sum(losses)/len(losses)
        progress_bar.update(1)
        progress_bar.set_description("avg loss so far = {}".format(avg_loss))


    if(epoch % 3 == 0):
      eval_ed_outputs = []
      eval_entity_labels = []
      eval_sentences = []
      bert.eval()
      ed_mapping_layer.eval()
      ed_dropout.eval()
      val_steps = int(len(val_samples)/batch_size)+1
      with tqdm(total=val_steps) as progress_bar:
        for i in range(0, len(val_samples), batch_size):

            with torch.no_grad():
              batch_sentences = [sample.sentence for sample in val_samples[i:i+batch_size]]
              batch_sentences = [sample.sentence for sample in val_samples[i:i+batch_size]]
              batch_ner_labels = [sample.iob_labels for sample in val_samples[i:i+batch_size]]
              batch_entities = [sample.entities for sample in val_samples[i:i+batch_size]]

              inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
              batch_ner_y = torch.LongTensor(inputs['labels']).cuda()
              batch_ed_label_y = torch.LongTensor(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map)).cuda()
              batch_ed_vector_y = get_entity_vectors(batch_ed_label_y,entity_vector_dict,int_to_entity_map).cuda()

              outputs = bert(
                    inputs['input_ids'].cuda(),
                    attention_mask=inputs['attention_mask'].cuda(),
                    token_type_ids=inputs['token_type_ids'].cuda()
                )


              sequence_output = ed_dropout(outputs[0])

              ed_logits = ed_mapping_layer(sequence_output)
              ed_logits = tanh(ed_logits)
              
              eval_sentences.extend(batch_sentences)
              eval_ed_outputs.extend(ed_logits)
              eval_entity_labels.extend(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map))
              progress_bar.update(1)
      distances = ED_avg_cos_dis(eval_entity_labels,eval_ed_outputs)
      print('Distances: ',np.mean(distances))
      true_labels,entity_predictions = ED_eval(eval_entity_labels,eval_ed_outputs,val_candidate_entities)
      print('acc: ',accuracy_score(true_labels,entity_predictions))

## eval ED model

### validation

In [ ]:
eval_ed_outputs = []
eval_entity_labels = []
eval_sentences = []
bert.eval()
ed_mapping_layer.eval()
ed_dropout.eval()
val_steps = int(len(val_samples)/batch_size)+1
with tqdm(total=val_steps) as progress_bar:
  for i in range(0, len(val_samples), batch_size):

      with torch.no_grad():
        batch_sentences = [sample.sentence for sample in val_samples[i:i+batch_size]]
        batch_sentences = [sample.sentence for sample in val_samples[i:i+batch_size]]
        batch_ner_labels = [sample.iob_labels for sample in val_samples[i:i+batch_size]]
        batch_entities = [sample.entities for sample in val_samples[i:i+batch_size]]

        inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
        batch_ner_y = torch.LongTensor(inputs['labels']).cuda()
        batch_ed_label_y = torch.LongTensor(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map)).cuda()
        batch_ed_vector_y = get_entity_vectors(batch_ed_label_y,entity_vector_dict,int_to_entity_map).cuda()

        outputs = bert(
              inputs['input_ids'].cuda(),
              attention_mask=inputs['attention_mask'].cuda(),
              token_type_ids=inputs['token_type_ids'].cuda()
          )


        sequence_output = ed_dropout(outputs[0])

        ed_logits = ed_mapping_layer(sequence_output)
        eval_sentences.extend(batch_sentences)
        eval_ed_outputs.extend(ed_logits)
        eval_entity_labels.extend(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map))
        progress_bar.update(1)
distances = ED_avg_cos_dis(eval_entity_labels,eval_ed_outputs)
np.mean(distances)

 99%|█████████▊| 68/69 [00:12<00:00,  5.52it/s]


0.17449264497267014

### test

In [ ]:
eval_ed_outputs = []
eval_entity_labels = []
eval_sentences = []
bert.eval()
ed_mapping_layer.eval()
ed_dropout.eval()
test_steps = int(len(test_samples)/batch_size)+1
with tqdm(total=test_steps) as progress_bar:
  for i in range(0, len(test_samples), batch_size):

      with torch.no_grad():
        batch_sentences = [sample.sentence for sample in test_samples[i:i+batch_size]]
        batch_sentences = [sample.sentence for sample in test_samples[i:i+batch_size]]
        batch_ner_labels = [sample.iob_labels for sample in test_samples[i:i+batch_size]]
        batch_entities = [sample.entities for sample in test_samples[i:i+batch_size]]

        inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
        batch_ner_y = torch.LongTensor(inputs['labels']).cuda()
        batch_ed_label_y = torch.LongTensor(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map)).cuda()
        batch_ed_vector_y = get_entity_vectors(batch_ed_label_y,entity_vector_dict,int_to_entity_map).cuda()

        outputs = bert(
              inputs['input_ids'].cuda(),
              attention_mask=inputs['attention_mask'].cuda(),
              token_type_ids=inputs['token_type_ids'].cuda()
          )


        sequence_output = ed_dropout(outputs[0])

        ed_logits = ed_mapping_layer(sequence_output)
        eval_sentences.extend(batch_sentences)
        eval_ed_outputs.extend(ed_logits)
        eval_entity_labels.extend(get_aligned_entity_labels(inputs['labels'],batch_entities,entity_to_int_map))
        progress_bar.update(1)
distances = ED_avg_cos_dis(eval_entity_labels,eval_ed_outputs)
np.mean(distances)

100%|██████████| 67/67 [00:11<00:00,  5.61it/s]


0.22119966606400043

In [ ]:
true_labels,entity_predictions = ED_eval(eval_entity_labels,eval_ed_outputs,test_candidate_entities)
accuracy_score(true_labels,entity_predictions)


import pickle

with open ('drive/My Drive/train_candidate_entities10.pkl', 'rb') as fp:
    train_candidate_entities = pickle.load(fp)

with open ('drive/My Drive/val_candidate_entities10.pkl', 'rb') as fp:
    val_candidate_entities = pickle.load(fp)

with open ('drive/My Drive/test_candidate_entities10.pkl', 'rb') as fp:
    test_candidate_entities = pickle.load(fp)

0.7024793388429752

In [ ]:
true_labels,entity_predictions = ED_eval(eval_entity_labels,eval_ed_outputs,val_candidate_entities)

In [ ]:
true_labels,entity_predictions = ED_eval(eval_entity_labels,eval_ed_outputs,train_candidate_entities)

 64%|██████▍   | 11857/18445 [00:13<00:07, 857.12it/s]

NO CANDIDATES CASE: 


100%|██████████| 18445/18445 [00:21<00:00, 861.27it/s]


In [ ]:
from sklearn.metrics import 

# f1_score(true_labels,entity_predictions,average='micro')
accuracy_score(true_labels,entity_predictions)


0.787434554973822

In [ ]:
# max_candidates = 1000
from kb.wiki_linking_util import WikiCandidateMentionGenerator
candidate_generator = WikiCandidateMentionGenerator(entity_world_path = None)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
100%|██████████| 1429887384/1429887384 [01:15<00:00, 19011080.21B/s]


duplicate_mentions_cnt:  6777
end of p_e_m reading. wall time: 1.1392328103383382  minutes
p_e_m_errors:  0
incompatible_ent_ids:  0


In [ ]:
MAX_CANDIDATES = 30

train_candidate_entities = get_dataset_entity_candidates(train_samples,candidate_generator,ebert_emb,MAX_CANDIDATES)
val_candidate_entities = get_dataset_entity_candidates(val_samples,candidate_generator,ebert_emb,MAX_CANDIDATES)
test_candidate_entities = get_dataset_entity_candidates(test_samples,candidate_generator,ebert_emb,MAX_CANDIDATES)


In [ ]:
train_candidate_entities

In [ ]:
import pickle

with open ('drive/My Drive/train_candidate_entities50.pkl', 'rb') as fp:
    train_candidate_entities = pickle.load(fp)

with open ('drive/My Drive/val_candidate_entities50.pkl', 'rb') as fp:
    val_candidate_entities = pickle.load(fp)

with open ('drive/My Drive/test_candidate_entities50.pkl', 'rb') as fp:
    test_candidate_entities = pickle.load(fp)

In [ ]:
import pickle

with open('drive/My Drive/train_candidate_entities30.pkl', 'wb') as fp:
    pickle.dump(train_candidate_entities, fp)

with open('drive/My Drive/val_candidate_entities30.pkl', 'wb') as fp:
    pickle.dump(val_candidate_entities, fp)

with open('drive/My Drive/test_candidate_entities30.pkl', 'wb') as fp:
    pickle.dump(test_candidate_entities, fp)

# NER Model

In [ ]:
# IOB tags and X
num_labels = 3

hidden_dropout_prob = 0.1
hidden_size = 768

bert = BertModel.from_pretrained('bert-base-cased',add_pooling_layer=False).cuda()
ner_dropout = torch.nn.Dropout(hidden_dropout_prob).cuda()
ner_classifier = torch.nn.Linear(hidden_size, num_labels).cuda()


# bert = BertModel.from_pretrained('bert-base-cased',add_pooling_layer=False)
# ner_dropout = torch.nn.Dropout(hidden_dropout_prob)
# ner_classifier = torch.nn.Linear(hidden_size, num_labels)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
learning_rate = 2e-5
# TODO only optimizing classifcation layer
params_to_optimize = list(ner_classifier.parameters()) +list(bert.parameters())
optimizer = AdamW(params_to_optimize, lr=learning_rate)

In [ ]:
batch_size = 1
Epochs = 5


training_steps = int(len(train_samples)/batch_size)+1
losses = []
for epoch in range(Epochs):
  with tqdm(total=training_steps) as progress_bar:
    for i in range(0, len(train_samples), batch_size):
        bert.train()
        ner_classifier.train()
        ner_dropout.train()

        batch_sentences = [sample.sentence for sample in train_samples[i:i+batch_size]]
        batch_ner_labels = [sample.iob_labels for sample in train_samples[i:i+batch_size]]

        inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
        # batch_ner_y = torch.LongTensor(inputs['labels'])
        batch_ner_y = torch.LongTensor(inputs['labels']).cuda()


        outputs = bert(
              inputs['input_ids'].cuda(),
              attention_mask=inputs['attention_mask'].cuda(),
              token_type_ids=inputs['token_type_ids'].cuda()
          )

        
        sequence_output = ner_dropout(outputs[0])
        logits = ner_classifier(sequence_output)

        loss = None
        if batch_ner_y is not None:
          loss_fct = torch.nn.CrossEntropyLoss().cuda()
          # loss_fct = torch.nn.CrossEntropyLoss()

          # Only keep active parts of the loss
          if inputs['attention_mask'].cuda() is not None:
            active_loss = inputs['attention_mask'].cuda().view(-1) == 1
            active_logits = logits.view(-1, num_labels)
            active_labels = torch.where(
                active_loss, batch_ner_y.view(-1), torch.tensor(loss_fct.ignore_index).type_as(batch_ner_y)
            )
            loss = loss_fct(active_logits, active_labels)
          else:
            loss = loss_fct(logits.view(-1, num_labels), batch_ner_y.view(-1))

        # TODO should I clip bert aswell?
        torch.nn.utils.clip_grad_norm_(ner_classifier.parameters(), 1.0)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        losses.append(loss.item())
        avg_loss = sum(losses)/len(losses)
        progress_bar.update(1)
        progress_bar.set_description("avg loss so far = {}".format(avg_loss))

    # TODO as function
    bert.eval()
    ner_classifier.eval()
    ner_dropout.eval()
    preds = []
    eval_labels = []

    val_steps = int(len(val_samples)/batch_size)+1

    
    for i in range(0, len(val_samples), batch_size):
        with torch.no_grad():

          batch_sentences = [sample.sentence for sample in val_samples[i:i+batch_size]]
          batch_ner_labels = [sample.iob_labels for sample in val_samples[i:i+batch_size]]
          # 

          inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
          batch_ner_y = torch.LongTensor(inputs['labels']).cuda()



          outputs = bert(
                inputs['input_ids'].cuda(),
                attention_mask=inputs['attention_mask'].cuda(),
                token_type_ids=inputs['token_type_ids'].cuda()
            )
          
          sequence_output = outputs[0]
          logits = ner_classifier(sequence_output)

          eval_labels.extend(batch_ner_y)
          preds.extend(torch.argmax(logits.softmax(dim = 1),axis=2))
    eval_preds(eval_labels,preds)


avg loss so far = 0.011383408278267249: 100%|█████████▉| 1094/1095 [01:50<00:00, 11.04it/s]


0.8825801467748164


avg loss so far = 0.011365195396206052:   0%|          | 2/1095 [00:00<01:43, 10.56it/s]

              precision    recall  f1-score   support

           _       0.82      0.96      0.88      4775

   micro avg       0.82      0.96      0.88      4775
   macro avg       0.82      0.96      0.88      4775
weighted avg       0.82      0.96      0.88      4775




avg loss so far = 0.009357231634921297: 100%|█████████▉| 1094/1095 [01:50<00:00, 11.20it/s]

...

              precision    recall  f1-score   support

           _       0.85      0.96      0.90      4775

   micro avg       0.85      0.96      0.90      4775
   macro avg       0.85      0.96      0.90      4775
weighted avg       0.85      0.96      0.90      4775




avg loss so far = 0.006060805372652556: 100%|█████████▉| 1094/1095 [01:50<00:00, 10.82it/s]


0.886999713439679


avg loss so far = 0.006060805372652556: 100%|█████████▉| 1094/1095 [02:07<00:00,  8.59it/s]

              precision    recall  f1-score   support

           _       0.82      0.97      0.89      4775

   micro avg       0.82      0.97      0.89      4775
   macro avg       0.82      0.97      0.89      4775
weighted avg       0.82      0.97      0.89      4775




In [ ]:
# TODO as function
bert.eval()
ner_classifier.eval()
ner_dropout.eval()
preds = []
eval_labels = []

val_steps = int(len(test_samples)/batch_size)+1


for i in range(0, len(test_samples), batch_size):
    with torch.no_grad():

      batch_sentences = [sample.sentence for sample in test_samples[i:i+batch_size]]
      batch_ner_labels = [sample.iob_labels for sample in test_samples[i:i+batch_size]]
      # 

      inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
      batch_ner_y = torch.LongTensor(inputs['labels']).cuda()



      outputs = bert(
            inputs['input_ids'].cuda(),
            attention_mask=inputs['attention_mask'].cuda(),
            token_type_ids=inputs['token_type_ids'].cuda()
        )
      
      sequence_output = outputs[0]
      logits = ner_classifier(sequence_output)

      eval_labels.extend(batch_ner_y)
      preds.extend(torch.argmax(logits.softmax(dim = 1),axis=2))
eval_preds(eval_labels,preds)


0.865811698467313
              precision    recall  f1-score   support

           _       0.81      0.93      0.87      4477

   micro avg       0.81      0.93      0.87      4477
   macro avg       0.81      0.93      0.87      4477
weighted avg       0.81      0.93      0.87      4477




# Additional NER training

In [ ]:
!git clone https://github.com/juand-r/entity-recognition-datasets.git

Cloning into 'entity-recognition-datasets'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 1044 (delta 2), reused 0 (delta 0), pack-reused 1038
Receiving objects: 100% (1044/1044), 2.43 MiB | 2.28 MiB/s, done.
Resolving deltas: 100% (421/421), done.


In [ ]:
import sys

sys.path += ['entity-recognition-datasets/src']

In [ ]:
import os 
import utils
IOB = "IO"
sep = "multispace"
iob_pos = 1
word_pos = 0
tag_placeholder = "TAGPLACEHOLDER"
root = "/content/entity-recognition-datasets/data/wikigold/CONLL-format/data/"
filename = "wikigold.conll.txt"
NER_DATASET = []
with open(os.path.join(root, filename), 'rb') as file_handle:
    try:
        file_content = file_handle.read().decode('utf-8').strip()
    except:
        raise ValueError("Can't process!")
    # Split sentences:
    annotated_sentences = file_content.split('\n\n')

    for annotated_sentence in annotated_sentences:
        if annotated_sentence not in ['-DOCSTART- -X- O O', '-DOCSTART- -X- -X- O']:
            # Split words:
            annotated_tokens = [seq for seq in annotated_sentence.split('\n')]
            standard_form_tokens = []

            for idx, annotated_token in enumerate(annotated_tokens):
                if sep=='multispace':
                    annotations = annotated_token.split()   # Split annotations
                else:
                    annotations = annotated_token.split(sep)   # Split annotations
                try:
                    word,tag ,ner = annotations[word_pos], tag_placeholder, annotations[iob_pos]
                except:
                    print(annotations)
                    #raise ValueError("??")
                if IOB =='IO':
                # Transform to IOB format if it is not already.
                    if ner != 'O':
                        ner = ner.split('-')[1]


                    standard_form_tokens.append((word, tag, ner))
                    conll_tokens = utils.to_conll_iob(standard_form_tokens)
                elif IOB == 'IOB1':
                    standard_form_tokens.append( (word,tag, ner))
                    conll_tokens = iob1_to_iob2(standard_form_tokens)
                elif IOB == 'none':
                    standard_form_tokens.append((word, tag, ner))
                    conll_tokens = to_conll_iob(standard_form_tokens)
                elif IOB == 'IOB2':
                    # This is for the Seminars_and_Job_postings
                    # data:
                    if ner=='0':
                        ner = 'O'
                    standard_form_tokens.append((word, tag, ner))
                    conll_tokens = standard_form_tokens
                else:
                    raise ValueError('Variable IOB has wrong value.')

            NER_DATASET.append([(w, iob) for w, t, iob in conll_tokens])

In [ ]:
documents= []
document = []
for sentence in NER_DATASET:
  for token in sentence:
    if("DOCSTART" in token[0] or len(document) > 350):
      documents.append(document)
      document = []
    else:
      document.append(token)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val = train_test_split(documents, test_size=0.1, random_state=42)

In [ ]:
X_train

In [ ]:
ner_train_samples = []

for doc in X_train:
  doc_iob = []
  doc_sentences = []
  for word, label in doc:
    doc_iob.append(label[0])
    doc_sentences.append(word)
  
  ner_train_samples.append(Sample(doc_sentences,doc_iob,None,None))



ner_val_samples = []

for doc in X_val:
  doc_iob = []
  doc_sentences = []
  for word, label in doc:
    doc_iob.append(label[0])
    doc_sentences.append(word)
  
  ner_val_samples.append(Sample(doc_sentences,doc_iob,None,None))

In [ ]:
full_set_samples=[]

for doc in documents:
  doc_iob = []
  doc_sentences = []
  for word, label in doc:
    doc_iob.append(label[0])
    doc_sentences.append(word)
  
  full_set_samples.append(Sample(doc_sentences,doc_iob,None,None))

In [ ]:
# IOB tags and X
num_labels = 3

hidden_dropout_prob = 0.1
hidden_size = 768

bert = BertModel.from_pretrained('bert-base-cased',add_pooling_layer=False).cuda()
ner_dropout = torch.nn.Dropout(hidden_dropout_prob).cuda()
ner_classifier = torch.nn.Linear(hidden_size, num_labels).cuda()


# bert = BertModel.from_pretrained('bert-base-cased',add_pooling_layer=False)
# ner_dropout = torch.nn.Dropout(hidden_dropout_prob)
# ner_classifier = torch.nn.Linear(hidden_size, num_labels)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
learning_rate = 2e-5
# TODO only optimizing classifcation layer
params_to_optimize = list(ner_classifier.parameters()) +list(bert.parameters())
optimizer = AdamW(params_to_optimize, lr=learning_rate)

In [ ]:
batch_size = 4
Epochs = 1


training_steps = int(len(full_set_samples)/batch_size)+1
losses = []
for epoch in range(Epochs):
  with tqdm(total=training_steps) as progress_bar:
    for i in range(0, len(full_set_samples), batch_size):
        bert.train()
        ner_classifier.train()
        ner_dropout.train()

        batch_sentences = [sample.sentence for sample in full_set_samples[i:i+batch_size]]
        batch_ner_labels = [sample.iob_labels for sample in full_set_samples[i:i+batch_size]]

        inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
        # batch_ner_y = torch.LongTensor(inputs['labels'])
        batch_ner_y = torch.LongTensor(inputs['labels']).cuda()


        outputs = bert(
              inputs['input_ids'].cuda(),
              attention_mask=inputs['attention_mask'].cuda(),
              token_type_ids=inputs['token_type_ids'].cuda()
          )

        
        sequence_output = ner_dropout(outputs[0])
        logits = ner_classifier(sequence_output)

        loss = None
        if batch_ner_y is not None:
          loss_fct = torch.nn.CrossEntropyLoss().cuda()
          # loss_fct = torch.nn.CrossEntropyLoss()

          # Only keep active parts of the loss
          if inputs['attention_mask'].cuda() is not None:
            active_loss = inputs['attention_mask'].cuda().view(-1) == 1
            active_logits = logits.view(-1, num_labels)
            active_labels = torch.where(
                active_loss, batch_ner_y.view(-1), torch.tensor(loss_fct.ignore_index).type_as(batch_ner_y)
            )
            loss = loss_fct(active_logits, active_labels)
          else:
            loss = loss_fct(logits.view(-1, num_labels), batch_ner_y.view(-1))

        # TODO should I clip bert aswell?
        torch.nn.utils.clip_grad_norm_(ner_classifier.parameters(), 1.0)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        losses.append(loss.item())
        avg_loss = sum(losses)/len(losses)
        progress_bar.update(1)
        progress_bar.set_description("avg loss so far = {}".format(avg_loss))

    # TODO as function
    bert.eval()
    ner_classifier.eval()
    ner_dropout.eval()
    preds = []
    eval_labels = []

    val_steps = int(len(val_samples)/batch_size)+1

    
    for i in range(0, len(val_samples), batch_size):
        with torch.no_grad():

          batch_sentences = [sample.sentence for sample in val_samples[i:i+batch_size]]
          batch_ner_labels = [sample.iob_labels for sample in val_samples[i:i+batch_size]]
          # 

          inputs = tokenize_and_align_labels(batch_sentences,batch_ner_labels,tokenizer)
          batch_ner_y = torch.LongTensor(inputs['labels']).cuda()



          outputs = bert(
                inputs['input_ids'].cuda(),
                attention_mask=inputs['attention_mask'].cuda(),
                token_type_ids=inputs['token_type_ids'].cuda()
            )
          
          sequence_output = outputs[0]
          logits = ner_classifier(sequence_output)

          eval_labels.extend(batch_ner_y)
          preds.extend(torch.argmax(logits.softmax(dim = 1),axis=2))
    eval_preds(eval_labels,preds)


avg loss so far = 0.38119591524203617:  98%|█████████▊| 51/52 [00:13<00:00,  3.71it/s]


0.511264367816092


  0%|          | 0/52 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           _       0.40      0.70      0.51      4775

   micro avg       0.40      0.70      0.51      4775
   macro avg       0.40      0.70      0.51      4775
weighted avg       0.40      0.70      0.51      4775




avg loss so far = 0.2305093051536995:  98%|█████████▊| 51/52 [00:13<00:00,  3.76it/s] 

...

              precision    recall  f1-score   support

           _       0.70      0.90      0.79      4775

   micro avg       0.70      0.90      0.79      4775
   macro avg       0.70      0.90      0.79      4775
weighted avg       0.70      0.90      0.79      4775




avg loss so far = 0.03346292696614256:  98%|█████████▊| 51/52 [00:13<00:00,  3.82it/s] 


0.794965034965035


avg loss so far = 0.03346292696614256:  98%|█████████▊| 51/52 [00:40<00:00,  1.27it/s]

              precision    recall  f1-score   support

           _       0.72      0.89      0.79      4775

   micro avg       0.72      0.89      0.79      4775
   macro avg       0.72      0.89      0.79      4775
weighted avg       0.72      0.89      0.79      4775




# NER Experiment


In [ ]:
! pip install simpletransformers

In [ ]:
import logging

import pandas as pd
from simpletransformers.ner import NERModel, NERArgs


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
train_df_rows = []
for i,sample in enumerate(train_samples):
  for token,label in zip(sample.sentence,sample.iob_labels):
    train_df_rows.append([i,token,label])


val_df_rows = []
for i,sample in enumerate(val_samples):
  for token,label in zip(sample.sentence,sample.iob_labels):
    val_df_rows.append([i,token,label])

test_df_rows = []
for i,sample in enumerate(test_samples):
  for token,label in zip(sample.sentence,sample.iob_labels):
    test_df_rows.append([i,token,label])

In [ ]:
train_df = pd.DataFrame(
    train_df_rows, columns=["sentence_id", "words", "labels"]
)

val_df = pd.DataFrame(
    val_df_rows, columns=["sentence_id", "words", "labels"]
)

test_df = pd.DataFrame(
    test_df_rows, columns=["sentence_id", "words", "labels"]
)

In [ ]:
model_args = NERArgs()
model_args.train_batch_size = 4
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.max_seq_length = 512
model_args.num_train_epochs = 25
model_args.overwrite_output_dir = True


labels_list = ["B", "I", "O"]
model_args.labels_list=labels_list


model = NERModel(
    "bert", "bert-base-cased" , args = model_args
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [ ]:
# Train the model
model.train_model(train_df, eval_data=val_df)

# Evaluate the model
result, model_outputs, preds_list = model.eval_model(test_df)

INFO:simpletransformers.ner.ner_model: Converting to features started.


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.08546631520285326, 'precision': 0.8032085561497326, 'recall': 0.9436649214659686, 'f1_score': 0.8677900818488206}


INFO:simpletransformers.ner.ner_model: Converting to features started.


...

INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.1797802683842533, 'precision': 0.8937648927720413, 'recall': 0.9426178010471205, 'f1_score': 0.9175415350117216}


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.18083475343883038, 'precision': 0.8934946449821499, 'recall': 0.9434554973821989, 'f1_score': 0.9177956605887746}


INFO:simpletransformers.ner.ner_model: Training of bert model complete. Saved to outputs/.
INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.2536451982005554, 'precision': 0.8457237523296749, 'recall': 0.912218003127094, 'f1_score': 0.8777133032452181}


# creating all possible mentions andcandidate for all sentences based of https://github.com/NPoe/ebert

In [ ]:
wikipedia2vec_file = "drive/MyDrive/enwiki_20180420_win10_100d.pkl"

In [ ]:
!wget http://wikipedia2vec.s3.amazonaws.com/models/en/2018-04-20/enwiki_20180420_win10_100d.txt.bz2


--2020-12-11 06:20:10--  http://wikipedia2vec.s3.amazonaws.com/models/en/2018-04-20/enwiki_20180420_win10_100d.txt.bz2
Resolving wikipedia2vec.s3.amazonaws.com (wikipedia2vec.s3.amazonaws.com)... 52.219.0.49
Connecting to wikipedia2vec.s3.amazonaws.com (wikipedia2vec.s3.amazonaws.com)|52.219.0.49|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 930773741 (888M) [application/x-bzip2]
Saving to: ‘enwiki_20180420_win10_100d.txt.bz2’

enwiki_20180420_win 100%[===================>] 887.65M  33.1MB/s    in 28s     

2020-12-11 06:20:38 (32.0 MB/s) - ‘enwiki_20180420_win10_100d.txt.bz2’ saved [930773741/930773741]



In [ ]:
!bzip2 -d enwiki_20180420_win10_100d.txt.bz2

In [ ]:
entity_vectors = []
with open('enwiki_20180420_win10_100d.txt','r') as f:
  for line in f:
    if("ENTITY/" in line):
      entity_vectors.append(line)


In [ ]:
entity_vector_dict = {}
for e in entity_vectors:
  title = e.split(' ')[0][7:]
  if(title in candidate_set or title in gold_candidate_set):
    vector_string = np.array(e.split(' ')[1:])
    vector = vector_string.astype(np.float)
    entity_vector_dict[title] = vector


In [ ]:
len(entity_vector_dict)

75059

In [ ]:
for e in entity_vectors:
  if '1994_Czech_Republic_motorcycle_Grand_Prix' in e:
    print(e) 
# 1994_Czech_Republic_motorcycle_Grand_Prix

In [ ]:
len(entity_vectors)

2592608

In [ ]:
x = np.array(entity_vectors[0].split()[1:])
y = x.astype(np.float)


In [ ]:
from wikipedia2vec import Wikipedia2Vec

wiki2vec = Wikipedia2Vec.load(wikipedia2vec_file)

In [ ]:
wiki2vec.get_entity_vector('City_of_Chester_(UK_Parliament_constituency)'.replace('_',' ')).shape

(100,)

In [ ]:
entity = all_old_entities[2]
entity.count

1912

In [ ]:
import sys
sys.getsizeof(wiki2vec.get_entity_vector('City_of_Chester_(UK_Parliament_constituency)'.replace('_',' '))) * 100000 /1024/1024

11.444091796875

In [ ]:
from allennlp.data.dataset_readers.dataset_utils import enumerate_spans

In [ ]:
sentence = "hello hat is this".split()
spans = enumerate_spans(sentence,max_span_width=3)
for span in spans:
  # print(span[0])
  print(sentence[span[0]:span[1] + 1])

['hello']
['hello', 'hat']
['hello', 'hat', 'is']
['hat']
['hat', 'is']
['hat', 'is', 'this']
['is']
['is', 'this']
['this']


In [ ]:

samples = []
ent2idx = {None: 0}

for data_idx, (sentence, spans) in tqdm(val_data.items(), disable = False, desc = "Converting data to samples"):
    mentions = candidate_generator.get_mentions_raw_text(" ".join(sentence), whitespace_tokenize=True)
    span2candidates = {}

    for (start, end), entities, priors in zip(mentions["candidate_spans"], mentions["candidate_entities"], mentions["candidate_entity_priors"]):
        if any([x.startswith(ent_prefix) or x == "[PAD]" for x in sentence[start:end+1]]):
            continue


        # normalized = [normalize_entity(self.ebert_emb, entity, self.ent_prefix) for entity in entities]

        valid_entities = []
        valid_entities_set = set()
        valid_priors = []

        for entity, prior in zip(entities, priors):
            if entity is None or entity in valid_entities_set:
                continue
            
            valid_entities.append(entity)
            valid_priors.append(prior)
            valid_entities_set.add(entity)

        biases = [None] + [np.log(x) for x in valid_priors]
        entities = [None] + valid_entities

        if len(entities) > 1:
            span2candidates[(start, end)] = (entities, biases)
    
    span2gold = {(start, end): entity for entity, start, end in spans}
         
    for (start, end), (candidates, biases) in span2candidates.items():
        assert not ("[CLS]" in sentence or "[MASK]" in sentence or "[SEP]" in sentence or "[UNK]" in sentence)

        gold_candidate = span2gold.get((start, end), None)
        if not gold_candidate in candidates:
            continue 
        
        correct_idx = candidates.index(gold_candidate)
        
        for entity in candidates:
            if not entity in ent2idx:
                ent2idx[entity] = len(ent2idx)
        
        sentence_with_pattern = sentence[:start] + left_pattern + sentence[start:end+1] + right_pattern + sentence[end+1:]

        present_entities = [token for token in sentence_with_pattern if token.startswith(ent_prefix)]
        sentence_with_pattern_ent = ["[UNK]" if token.startswith(ent_prefix) else token for token in sentence_with_pattern]
        sample_tokenized = tokenizer.tokenize(" ".join(sentence_with_pattern_ent))

        ent_pos = [i for i, token in enumerate(sample_tokenized) if token == "[UNK]"]
        del_pos = [i for i, token in enumerate(sample_tokenized) if token == "[PAD]"]

        assert len(ent_pos) == len(present_entities)
        for pos, ent in zip(ent_pos, present_entities):
            sample_tokenized[pos] = ent

        for pos in sorted(del_pos, reverse = True):
            del sample_tokenized[pos]

        # if not self.do_use_priors:
        #     biases = None

        mask_pos = sample_tokenized.index("[MASK]")

        sample_tokenized = ["[CLS]"] + sample_tokenized + ["[SEP]"]
        input_ids = tokenizer.convert_tokens_to_ids(sample_tokenized)

        samples.append(Sample(input_ids = input_ids, 
            tokenized = sample_tokenized,
            mask_pos = sample_tokenized.index("[MASK]"),
            correct_idx = correct_idx,
            candidate_ids = [ent2idx[candidate] for candidate in candidates],
            biases = biases,
            sentence = sentence,
            candidates = candidates,
            start = start, end = end, data_idx = data_idx))

# if verbose:
#     for i in rnd.randint(low = 0, high = len(samples), size = (5,)):
#         print(samples[i], flush = True)


Converting data to samples: 100%|██████████| 272/272 [02:17<00:00,  1.97it/s]


In [ ]:
# max_candidates = 1000
from kb.wiki_linking_util import WikiCandidateMentionGenerator
candidate_generator = WikiCandidateMentionGenerator(entity_world_path = None)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
100%|██████████| 1429887384/1429887384 [00:36<00:00, 39576348.08B/s]


duplicate_mentions_cnt:  6777
end of p_e_m reading. wall time: 1.143068301677704  minutes
p_e_m_errors:  0
incompatible_ent_ids:  0


In [ ]:
candidate_generator.get_mentions_raw_text('Chester Bennington of LP')

In [ ]:
## apply this for spans
candidate_generator.process('Chester City F.C.')

[]

In [ ]:
from wikipedia2vec import Wikipedia2Vec

wiki2vec = Wikipedia2Vec.load('drive/My Drive/wikipedia2vec-base-cased')

In [ ]:
from wikipedia2vec import Wikipedia2Vec

wiki2vec = Wikipedia2Vec.load('drive/My Drive/enwiki_20180420_win10_100d.pkl')